# 一次元躯体構成群から有効熱容量を計算するプログラム

- 一次元壁体構成から周波数応答を経て有効熱容量を計算
  - 住戸全体、複数空間、単一空間等の有効熱容量を計算する空間に面する躯体の構成、熱物性、面積等を入力
  - 計算対象外の隣接する空間(外気、隣接住戸、床下他)の温度差係数を加味
  - 対象空間に面する躯体の有効熱容量、室内側吸熱熱容量、貫流熱容量を算出




## 1. 本体プログラム



### 1.1 入力値と出力値

#### (1) 有効熱容量を求める空間単位と面する一次元躯体の設定方法

##### a) 対象とする空間単位
有効熱容量を求める空間は以下の単位とする。

- 住戸全体：計算対象住戸の居室、非居室と断熱区画内の空間
- 複数の空間：計算対象とする住戸の複数の居室、非居室と断熱区画内の空間
- 単一の空間：計算対象とする住戸の単一の居室、非居室と断熱区画内の空間

上記の空間(群)が面する、外界に面する躯体、隣接住戸の境界となる躯体、同一住戸内の躯体の総数を$M$($m=1$～$M$)とする。

##### b) 躯体の設定

躯体$M$について、以下の値を設定する。なお、外界に面していない躯体については、2つの表面のそれぞれの方向から別個に入力することはないものとする(例：同一住戸の1、2階間の床では、2階を$n=0$側、1階を$n=N$側(隣接する空間側)とし、壁の種別(`Wtype`)を設定することで、隣接側の励振を考慮するようにする)。

- 表面積$A_m \; [m^2]$
- 壁の種別`Wtype`：室内側有効吸熱熱容量、有効貫流熱容量の加算対象かどうかを判別するためのタイプ分け
  - `Wtype` $=0$：外界に面した躯体、隣接住戸の境界となる躯体(=隣接空間$n=N$側は対象空間ではない) → 有効貫流熱容量の加算対象。$U_A$値計算対象(面積、熱貫流率積算対象)。
  - `Wtype` $=1$：両面が対象とする空間に面した躯体 → $n=0$、$n=N$の両側が室内側有効吸熱熱容量の加算対象。有効貫流熱容量には加算されない。
  - `Wtype` $=2$：隣接空間側が対象空間外だが同一住戸等 → $n=0$側が室内側有効吸熱熱容量の加算対象。$n=N$側の室内側有効吸熱熱容量は隣接空間側で加算されるべきとして加算対象外。有効貫流熱容量には加算対象外。


- $n=N$側空間に適用する温度差係数$H_m \; [-]$ 
- $n=N$側空間の温度変動に適用する位相のずれ$T_{\psi,m} \; [hour]$
  - $T_{\psi,m} <0$：隣接空間の変動が対象空間より遅れる。
  - ただし、原則、位相のずれはないものとする → $T_{\psi,m}=0$
    - すなわち、隣接空間の温度変動として、対象空間の温度変動と同位相で温度差係数分小さい振幅の変動(振幅が$(1-H_m)$の振動)を考えることになる。

温度差係数$H_m$は、「平成28年省エネルギー基準における住宅のエネルギー消費性能の算定方法第二章第二節 外皮性能」のもと、隣接空間の種類に応じて、以下の値を設定する。ただし、「*」については暫定の設定。

- 外気および外気に通じる空間：$H_m=1.0$
- 外気に通じていない空間および外気に通じる床裏：$H_m=0.7$
- 住戸及び住戸と同様の熱的環境の空間、ならびに外気に通じていない床裏：

  - $1$～$3$地域：$H_m=0.05$
  - $4$～$8$地域：$H_m=0.15$


- 同一住戸内の同様の熱的環境の空間*：$H_m=0.0$
  - `Wtype` $=1, 2$で設定
  
また、土間床については、「平成25年省エネルギー基準に準拠した算定・判断の方法及び解説Ⅱ住宅(連合印刷センター,2013年)」の第三部第二節では、温度差係数として$H_m=0$が適用されている(要確認)。


##### c) 躯体の層構成

本稿で扱う一次元躯体$m$は、$N_m-2\;$層からなる躯体構成要素とその両面の表面熱伝達層を持つ、全体で$N_m$層で構成される躯体とする。層構成を以下のように割り当てる。

- 層$1 \; (n=1)$：振幅$1$の温度変動を適用する空間側(例：外壁の室内側)の表面熱伝達層
- 層$N_m \; (n=N_m)$：層$1$と反対側(例：外壁の外気側)の表面熱伝達層
- 層$2$～層$N_m-1 \; (n=2$～$N_m-1)$：躯体を構成する部材の層(中空層含む)($n=1$側から層順に設定)

一次元躯体$m$の層$n$には、以下の物性値等を設定する。

- 厚さ$d_{m,n}\;[m]$
- 熱伝導率$\lambda_{m,n} \; [W/(m･K)]$
- 容積比熱$(c\rho)_{m,n} \; [kJ/(m^3･K)]$

  - 表面熱伝達層や中空層の場合は、$(c\rho)_{m,n} = 0\;[kJ/(m^3･K)]$とする。

  - 中空層の場合は、厚さ$d_{m,n}\;[m]$を、その中空層の熱抵抗$r_{m,n} \; [m^2K/W]$で除した値を、$\lambda_{m,n} \; [W/(m･K)]$として設定する($\lambda_{m,n} \leftarrow d_{m,n} \: / \: r_{m,n} $)。

  - 表面熱伝達層の場合は、$d_{m,n}=1\;[m]$とし、表面熱伝達率$\alpha_s \; [W/(m^2･K)]$を$\lambda_{m,n}$に設定する($\lambda_{m,n} \leftarrow \alpha_s $)。


#### (2) 入力値

- 一次元躯体構成$m$の表面積$\; [m^2]$：$A_m$   ($m=1$～$M$)
- 一次元躯体構成$m$の壁の種別`Wtype(m)`   ($m=1$～$M$)
  - `Wtype`の設定値は(1)の通り。
  
  
- 躯体$m$を介して隣接する空間に適用する温度差係数$\; [-]$：$H_m$   ($m=1$～$M$)

  - 温度差係数の設定は(1)の通り。
  
  
- 躯体$m$を介して隣接する空間との温度変動の位相のずれ$\; [hour]$：$T_{\psi,m}$   ($m=1$～$M$)

    - ただし、原則、位相のずれはないものとする → $T_{\psi,m}=0$
  

- 一次元躯体構成$m$の層数：$N_m$   ($m=1$～$M$)
- 一次元躯体構成$m$の層$1$～層$N$の厚さ$\;[m]$：$d_{m,n}$   ($m=1$～$M$, $n=1$～$N_m$)
- 一次元躯体構成$m$の層$1$～層$N$の熱伝導率$\; [W/(m･K)]$：$\lambda_{m,n}$   ($m=1$～$M$, $n=1$～$N_m$)
- 一次元躯体構成$m$の層$1$～層$N$の容積比熱$\; [kJ/(m^3･K)]$：$(c\rho)_{m,n}$   ($m=1$～$M$, $n=1$～$N_m$)
- 周期$\; [hour]$：$T_i$    ($i=1$～)

    - ただし、原則、周期は$24$時間のみ → $T_1=24$
  


#### (3) 引数と入力値

`calc_specific_heat_capacity(AAA, Wtype, HHH, T_psi, NNN, d_ns, lambda_ns, c_rho_ns, TT)`において、
  
- 以下の$4$つの引数は、躯体構成数分($M$)分のリストを入力する。
  - `AAA` $ = [ A_1, \; A_2, \; \cdots, \; A_M ]$
  - `Wtype` $ = [ $ `Wtype(1)` $, \; $ `Wtype(2)` $, \; \cdots, \; $ `Wtype(M)` $]$
  - `HHH` $ = [ H_1, \; H_2, \; \cdots, \; H_M ]$
  - `T_psi` $ = [ T_{\psi,1}, \; T_{\psi,2}, \; \cdots, \; T_{\psi,M} ]$
  - `NNN` $ = [ N_1, \; N_2, \; \cdots, \; N_M ]$
    
    
- 以下の$3$つの引数は、層数分($N_m$)のリストを躯体構成数分($M$)結合したリスト
  - `d_ns` $ = [ d_{1,1}, \; d_{1,2}, \; \cdots, \; d_{1,N_1}, \; d_{2,1}, \; \cdots, \; d_{2,N_2},\; \cdots, \; d_{M,1}, \; \cdots, \; d_{M,N_M} \; ]$
  - `lambda_ns` $ = [ \lambda_{1,1}, \; \lambda_{1,2}, \; \cdots, \; \lambda_{1,N_1}, \; \lambda_{2,1}, \; \cdots, \; \lambda_{2,N_2},\; \cdots, \; \lambda_{M,1}, \; \cdots, \; \lambda_{M,N_M} \; ]$
  - `c_rho_ns` $ = [ (c\rho)_{1,1}, \; (c\rho)_{1,2}, \; \cdots, \; (c\rho)_{1,N_1}, \; (c\rho)_{2,1}, \; \cdots, \; (c\rho)_{2,N_2},\; \cdots, \; (c\rho)_{M,1}, \; \cdots, \; (c\rho)_{M,N_M} \; ]$ 
    
    
- 以下の引数は、計算する周期を列挙したリスト
  - `TT` $ = [ T_{1}, \; T_{2}, \; \cdots, \; ]$


#### (4) 出力値  ●●この項未修正あり●●

- 一次元躯体$m=1$～$M$の表面積(片側のみ)合計値$\; [m^2]$：$A_{total}$
- $U_A$値計算に用いる外皮表面積$\; [m^2]$：$A_{total1}$
- 一次元躯体$m=1$～$M$の熱容量合計値$\; [kJ/K]$：$C$ 
- 内外1℃差(温度差係数考慮)、定常状態での蓄熱量$\; [kJ/K]$：$C_{static}$ 
- $U_A$値$\; [W/(m^2K)]$：$U_A$
- 有効熱容量を計算する周期$\; [hour]$：$T_i$
- 周期$T_i$における一次元躯体$m=1$～$M$の有効熱容量合計値$\; [kJ/K]$：$C_{s}$ 


- 周期$T_i$における$a$側もしくは$b$側励振に対する$a$側もしくは$b$側励振の周波数応答の係数積算値$\; [W/K]$：
  - $|q_{a,a}| \cos \varphi_{a,a}$, $|q_{a,b}| \cos \varphi_{a,b}$, $|q_{b,a}| \cos \varphi_{b,a}$, $|q_{b,b}| \cos \varphi_{b,b}$
  - $|q_{a,a}| \sin \varphi_{a,a}$, $|q_{a,b}| \sin \varphi_{a,b}$, $|q_{b,a}| \sin \varphi_{b,a}$, $|q_{b,b}| \sin \varphi_{b,b}$


#### (5) 戻り値と出力値  ●●この項未修正あり●●

`[Atotal,Atotal1,CC,CCsta,UUA,TT,CCAI,CCT,CCI,QAIC,QAIS,QTC,QTS] = calc_specific_heat_capacity(AAA, Wtype, HHH, T_psi, NNN, d_ns, lambda_ns, c_rho_ns, TT)`において、

- `Atotal` $ = A_{total}$
- `Atotal1` $ = A_{total1}$
- `CC` $ = C$
- `CCsta` $ = C_{static}$ 
- `UUA` $ = U_A$
- `TT` $ = [ T_{1}, \; T_{2}, \; \cdots, \; ]$
- `QC` $ = [ |q_{a,a}| \cos \varphi_{a,a}, \; |q_{a,b}| \cos \varphi_{a,b}, \; |q_{b,a}| \cos \varphi_{b,a}, \; |q_{b,b}| \cos \varphi_{b,b}, \;  \cdots, \; $ (以下、計算対象周期分繰返し) $\: ]$
- `QS` $ = [ |q_{a,a}| \sin \varphi_{a,a}, \; |q_{a,b}| \sin \varphi_{a,b}, \; |q_{b,a}| \sin \varphi_{b,a}, \; |q_{b,b}| \sin \varphi_{b,b}, \;  \cdots, \; $ (以下、計算対象周期分繰返し) $\: ]$

### 1.2 計算の流れ


#### (1) 熱容量、熱貫流率の計算式

- 対象空間が面する躯体の熱容量$C \; [kJ/K]$の算定式：
  - 躯体の熱容量は、入力した躯体すべてについて熱容量として加算することに変更する。
    - 温度差係数は非考慮とし、隣接空間との温度差は考慮しない → 単純に熱容量としての評価とする。

$$ C = \sum_{m=1}^{M} \biggl( A_m \sum_{n=1}^{N_m} (c\rho)_{m,n} d_{m,n} \biggr) \qquad (1) \; (\leftarrow 1.5 式(a))$$

- $a$側の温度$1 \: [K]$、$b$側の温度$1-H_m \: [K]$の温度差$H_m$(外気とは$1K$差)の定常状態における蓄熱量$C_{static} \; [kJ/K]$の算定式：

$$
C_{static} =  \sum_{m=1}^{M} \biggl[ A_m \sum_{n=1}^{N_m} \biggl[ (c\rho)_{m,n} d_{m,n} \biggl\{ H_m - U_m H_m \biggl( \sum_{n'=1}^{n-1} \frac{ d_{m,n'} }{ \lambda_{m,n'} } + \frac{ d_{m,n} }{ 2 \lambda_{m,n} } \biggr) \biggr\} \biggr] \biggr] \qquad (2) \; (\leftarrow 1.5 式(e))$$

- ここで、$U_m$は熱貫流率$[W/(m^2K)]$:   
  $$ U_m = 1 \bigg/ \sum_{n=1}^{N_m} \frac{ d_{m,n} }{ \lambda_{m,n} } \qquad (2.1) \; (\leftarrow 1.5 式(b))$$

- $U_A$値(ただし、熱橋等の線熱貫流率非考慮)$[W/K]$の算定式：
  - `Wtype`$=0$の躯体について、熱貫流率を積和して求める。
  - $\delta_{wtype=0}\:$は、`Wtype`$=0$で$1$を、それ以外の`Wtype`で$0$をとる。

$$U_A = \sum_{m=1}^{M} \bigl( \delta_{wtype=0} \: A_m U_m H_m \bigr) \big/ \sum_{m=1}^{M} \bigl( \delta_{wtype=0} \:A_m \bigr) \qquad (2.2) \; (\leftarrow 1.5 式(e1))$$




#### (2) 周波数応答を計算する周期の設定と角振動数の計算式

- 原則として、有効熱容量を計算する周期は、$T=24\:[hour]$とする。

  - $24$時間以外の周期も受け付けられるようにする。
  - 角周波数$\omega \: [rad/s] \:$は、周期$T\: [hour]\: $から下式で求める。

$$\omega = 2 \pi \: \big/ \: (T \times 3600) \qquad (3) \; (\leftarrow 1.5 式(f))$$




#### (3) 単層の四端子行列の計算式

1.5 (4)をあわせて参照のこと

- 本稿内では、下記について、入力時と計算時で単位が異なる。中途で変換されるので注意すること。
   - 時間$T$：入力時は$[hour]$立て、計算時は基本的に角周波数$[rad/s]$を用いているので表出しないが$[s]$立てとなる。
   - 容積比熱$c\rho$：入力時は$[kJ/(m^3･K)]$、計算時は$[J/(m^3･K)]$。
   
##### a) 躯体単層の四端子行列

$(c\rho)_{m,n} \neq 0$の単層については、角周波数$\omega\:[rad/s]$、各層の厚さ$d_{m,n}\:[m]$、熱伝導率$\lambda_{m,n}\:[W/(m･K)]$、容積比熱$(c\rho)_{m,n}\:[J/(m^3･K)]$から層$n$の単層の四端子行列$\textbf{Z}_{m,n}$を算出する。$j$は虚数単位である($\:j=\sqrt{(-1)}\:$)。

- 各層の侵入厚さの比$\:\xi\:$：

$$ \xi_{m,n} = \frac{d_{m,n}}{\delta_{m,n}} = d_{m,n} \sqrt {\frac{\pi (c\rho)_{m,n}}{\lambda_{m,n} T}} = d_{m,n} \sqrt {\frac{\omega (c\rho)_{m,n}}{2 \lambda_{m,n}}} \qquad (4) \; (\leftarrow 1.5 式(13'))$$

- 躯体単層($\:(c\rho)_{m,n} > 0\:$)の四端子行列$\;\textbf{Z}_{m,n} = \begin{pmatrix} Z_{11,m,n} & Z_{12,m,n} \\ Z_{21,m,n} & Z_{22,m,n} \end{pmatrix}\;$ の要素：

$$ \begin{align}
Z_{11,m,n} = Z_{22,m,n} &= \cosh(\xi_{m,n}) \cos(\xi_{m,n}) + j \sinh(\xi_{m,n}) \sin(\xi_{m,n}) \\ 
Z_{12,m,n} &= - \frac{\delta_{m,n}}{2 \lambda_{m,n}} \{ \sinh(\xi_{m,n}) \cos(\xi_{m,n}) + \cosh(\xi_{m,n}) \sin(\xi_{m,n}) \\
& \hspace{30pt} + j \;[ \cosh(\xi_{m,n}) \sin(\xi_{m,n}) - \sinh(\xi_{m,n}) \cos(\xi_{m,n}) ] \} \\
Z_{21,m,n} &= - \frac{\lambda_{m,n}}{\delta_{m,n}} \{\sinh(\xi_{m,n}) \cos(\xi_{m,n}) - \cosh(\xi_{m,n}) \sin(\xi_{m,n})\\ 
& \hspace{30pt} + j \; [\sinh(\xi_{m,n}) \cos(\xi_{m,n}) + \cosh(\xi_{m,n}) \sin(\xi_{m,n})]\} \qquad (5) \; (\leftarrow 1.5 式(14'))
\end{align} $$


##### b) 表面熱伝達層、中空層の四端子行列

$(c\rho)_{m,n} = 0$の表面熱伝達層、中空層については、熱抵抗$R_{m,n} = d_{m,n}\:/\:\lambda_{m,n}$を用いて、下式で単層の四端子行列$\textbf{Z}_{m,n}$を算出する。

- 表面熱伝達層、中空層($\:(c\rho)_{m,n} = 0\:$)の四端子行列：
  
$$ \;\textbf{Z}_{m,n} = \begin{pmatrix} 1 & -R_{m,n} \\ 0 & 1 \end{pmatrix} = \begin{pmatrix} 1 & -d_{m,n}/\lambda_{m,n} \\ 0 & 1 \end{pmatrix} \qquad (6) \; (\leftarrow 1.5 式(15'), \; (18'))$$


#### (4) 一次元躯体の四端子行列の計算式

層$1$～$N_m$の四端子行列を、各層の行列をかけあわせて算出する。

- 層$1$～$N_m$の四端子行列$\;\textbf{Z}_m\;$：

$$\textbf{Z}_m  = \begin{pmatrix} Z_{11,m} & Z_{12,m} \\ Z_{21,m} & Z_{22,m} \end{pmatrix} = \textbf{Z}_{m,N} \; \textbf{Z}_{m,N-1} \;  \cdots \;  \textbf{Z}_{m,2} \;  \textbf{Z}_{m,1} \qquad (7) \; (\leftarrow 1.5 式(16'))$$


#### (5) 熱流の周波数応答の計算式

1.5 (6)をあわせて参照のこと

##### a) a側が単位温度振幅で励振する際の熱流応答

- $\hat{\theta}_{m,a} = \cos \omega t$、$\hat{\theta}_{m,b} = 0\:$の$a$側での単位温度励振の場合の熱流応答の係数の計算式：
  - $a$側励振の$a$側熱流応答：$\; \hat{q}_{m,a,a} = |{q}_{m,a,a}| \cos \varphi_{m,a,a} \cos ( \omega t ) - |{q}_{m,a,a}| \sin \varphi_{m,a,a} \sin ( \omega t ) \; $
  - $a$側励振の$b$側熱流応答：$\; \hat{q}_{m,a,b} = |{q}_{m,a,b}| \cos \varphi_{m,a,b} \cos ( \omega t ) - |{q}_{m,a,b}| \sin \varphi_{m,a,b} \sin ( \omega t ) \; $

$$
\left\{
\begin{array}{ll}
|{q}_{m,a,a}| \cos \varphi_{m,a,a} =  ( \: |Z_{11,m}| \: \big/ \: |Z_{12,m}|  \;) \; \cos \{ \arg(Z_{11,m}) - \arg(Z_{12,m}) + \pi \}
\\
|{q}_{m,a,a}| \sin \varphi_{m,a,a} =( \: |Z_{11,m}| \: \big/ \: |Z_{12,m}|  \;) \; \sin \{ \arg(Z_{11,m}) - \arg(Z_{12,m}) + \pi \}
\\
|{q}_{m,a,b}| \cos \varphi_{m,a,b} = ( \: 1 \: \big/ \: |Z_{12,m}|  \;) \; \cos \{ - \arg(Z_{12,m}) - \pi \}
\\
|{q}_{m,a,b}| \sin \varphi_{m,a,b} = ( \: 1 \: \big/ \: |Z_{12,m}|  \;) \; \sin \{ - \arg(Z_{12,m}) - \pi \}
\end{array}
\right.  \qquad (8) \; (\leftarrow 1.5 式(l))
$$

- ここで、$|Z_{**}|$は、$Z_{**}$の大きさ$\bigl( \:|Z_{**}| = \sqrt{\Re(Z_{**})^2 + \Im(Z_{**})^2} \: \bigr)$である。
- また、$\arg(Z_{**})$は、$Z_{**}$の偏角$\bigl( \: \arg(Z_{**}) = \tan^{-1} \{\Im(Z_{**}) \: \big/ \: \Re(Z_{**})\} \: \bigr)$である。

##### b) b側が励振する際の熱流応答

- $\hat{\theta}_{m,b} = (1 - H_m) \cos (\omega t + \psi_m' )$、$\hat{\theta}_{m,a} = 0\:$で$b$側で温度励振する場合の熱流応答の係数の計算式：
  - $\hat{\theta}_{m,b}$の位相のずれ：$\psi_m' = 2 \pi \: \big/ \: (T_{\psi,m} \times 3600) $
  - $b$側励振の$a$側熱流応答：$\; \hat{q}_{m,b,a} = |{q}_{m,b,a}| \cos \varphi_{m,b,a} \cos ( \omega t ) - |{q}_{m,b,a}| \sin \varphi_{m,b,a} \sin ( \omega t ) \; $
  - $b$側励振の$b$側熱流応答：$\; \hat{q}_{m,b,b} = |{q}_{m,b,b}| \cos \varphi_{m,b,b} \cos ( \omega t ) - |{q}_{m,b,b}| \sin \varphi_{m,b,b} \sin ( \omega t ) \; $

$$
\left\{
\begin{array}{ll}
|{q}_{m,b,a}| \cos \varphi_{m,b,a} = \{ \: (1 - H_m) \: \big/ \: |Z_{12,m}|  \; \} \cos \{ \psi_m' - \arg(Z_{12,m}) \}
\\
|{q}_{m,b,a}| \sin \varphi_{m,b,a} = \{ \: (1 - H_m) \: \big/ \: |Z_{12,m}|  \; \} \sin \{ \psi_m' - \arg(Z_{12,m}) \}
\\
|{q}_{m,b,b}| \cos \varphi_{m,b,b} = \{ \: (1 - H_m) \: |Z_{22,m}| \: \big/ \: |Z_{12,m}|  \; \} \cos \{ \psi_m' -  \arg(Z_{12,m}) + \arg(Z_{22,m}) \}
\\
|{q}_{m,b,b}| \sin \varphi_{m,b,b} = \{ \: (1 - H_m) \: |Z_{22,m}| \: \big/ \: |Z_{12,m}|  \; \} \sin \{ \psi_m' -  \arg(Z_{12,m}) + \arg(Z_{22,m}) \}
\\
\end{array}
\right.  \qquad (9) \; (\leftarrow 1.5 式(p))
$$

##### c) a側、b側が温度励振する時のa側、b側の熱流応答

- $\hat{\theta}_{m,a} = \cos \omega t$、$\hat{\theta}_{m,b} = (1 - H_m) \cos (\omega t + \psi_m' )$で温度変動する場合の、$a$側の熱流$\hat{q}_{m,a}$、$b$側の熱流$\hat{q}_{m,b}$の計算式：
  - $\hat{q}_{m,a} = \hat{q}_{m,a,a}+\hat{q}_{m,b,a} = |{q}_{m,a}| \cos \varphi_{m,a} \cos \omega t - |{q}_{m,a}| \sin \varphi_{m,a}  \sin \omega t$
  - $\hat{q}_{m,b} = \hat{q}_{m,a,b}+\hat{q}_{m,b,b} = |{q}_{m,b}| \cos \varphi_{m,b} \cos \omega t - |{q}_{m,b}| \sin \varphi_{m,b}  \sin \omega t$

$$
\left\{
\begin{array}{ll}
|{q}_{m,a}| \cos \varphi_{m,a} = |{q}_{m,a,a}| \cos \varphi_{m,a,a}+|{q}_{m,b,a}| \cos \varphi_{m,b,a}\\
|{q}_{m,a}| \sin \varphi_{m,a} = |{q}_{m,a,a}| \sin \varphi_{m,a,a}+|{q}_{m,b,a}| \sin \varphi_{m,b,a}\\
|{q}_{m,b}| \cos \varphi_{m,b} = |{q}_{m,a,b}| \cos \varphi_{m,a,b}+|{q}_{m,b,b}| \cos \varphi_{m,b,b}\\
|{q}_{m,b}| \sin \varphi_{m,b} = |{q}_{m,a,b}| \sin \varphi_{m,a,b}+|{q}_{m,b,b}| \sin \varphi_{m,b,b}\\
\end{array}
\right.  \qquad (10) \; (\leftarrow 1.5 式(r))
$$

##### d) 熱流の定常成分
- $\hat{\theta}_{m,a} = \cos \omega t$、$\hat{\theta}_{m,b} = (1 - H_m) \cos (\omega t + \psi_m' )$で温度変動する場合の熱流の定常成分$\hat{q}_{s,m}$の計算式：
  - $\hat{q}_{s,m} = U_m (\hat{\theta}_{m,a}-\hat{\theta}_{m,b}) = |{q}_{s,m}| \cos \varphi_{s,m} \cos \omega t - |{q}_{s,m}| \sin \varphi_{s,m} \sin \omega t $

$$
\left\{
\begin{array}{ll}
|{q}_{s,m}| \cos \varphi_{s,m} = U_m (1- \cos \psi_m'+ H_m \cos \psi_m' )\\
|{q}_{s,m}| \sin \varphi_{s,m} = U_m (1 - H_m) \sin \psi_m'\\
\end{array}
\right.  \qquad (11) \; (\leftarrow 1.5 式(s))
$$


#### (6) 有効熱容量算定のための係数の積算

1.5 (7)をあわせて参照のこと

- 対象とする空間についての有効熱容量${C}_{i}$、室内側吸熱有効熱容量${C}_{Ai}$、貫流有効熱容量${C}_{T}$を計算するために、躯体$m$($=1$～$M$)について、`Wtype`にもとづいて熱流の係数に面積をかけて積算する。
  - $\delta_{wtype=0}\:$は、`Wtype`$=0$で$1$を、それ以外の`Wtype`で$0$をとる。
  - $\delta_{wtype=1}\:$は、`Wtype`$=1$で$1$を、それ以外の`Wtype`で$0$をとる。
  - $\delta_{wtype \neq 2}\:$は、`Wtype`$=0$、`Wtype`$=1$で$1$を、`Wtype`$=2$で$0$をとる。

$$\begin{align}
|{Q}_{Ai}| \cos \varphi_{Ai} = \sum_{m=1}^{M} & A_m \:\bigl\{ |{q}_{m,a}| \cos \varphi_{m,a} - |{q}_{s,m}| \cos \varphi_{s,m} \\
&\hspace{12pt} + \delta_{wtype=1} ( |{q}_{s,m}| \cos \varphi_{s,m} - |{q}_{m,b}| \cos \varphi_{m,b} ) \bigr\} \qquad (12) \; (\leftarrow 1.5 式(t1)) \\
|{Q}_{Ai}| \sin \varphi_{Ai} = \sum_{m=1}^{M} & A_m \:\bigl\{ |{q}_{m,a}| \sin \varphi_{m,a} - |{q}_{s,m}| \sin \varphi_{s,m} \\
&\hspace{12pt} + \delta_{wtype=1} ( |{q}_{s,m}| \sin \varphi_{s,m} - |{q}_{m,b}| \sin \varphi_{m,b} ) \bigr\} \qquad (13) \; (\leftarrow 1.5 式(t2)) \\
|{Q}_{T}| \cos \varphi_{T} = \sum_{m=1}^{M} & \delta_{wtype=0} \: A_m \:( |{q}_{s,m}| \cos \varphi_{s,m} - |{q}_{m,b}| \cos \varphi_{m,b} ) \qquad (14) \; (\leftarrow 1.5 式(u1)) \\
|{Q}_{T}| \sin \varphi_{T} = \sum_{m=1}^{M} & \delta_{wtype=0} \: A_m \:( |{q}_{s,m}| \sin \varphi_{s,m} - |{q}_{m,b}| \sin \varphi_{m,b} ) \qquad (15) \; (\leftarrow 1.5 式(u2)) \\
|{Q}_{i}| \cos \varphi_{i} = \sum_{m=1}^{M} & A_m \:\bigl\{ |{q}_{m,a}| \cos \varphi_{m,a} - |{q}_{s,m}| \cos \varphi_{s,m} \\
&\hspace{12pt} + \delta_{wtype \neq 2} ( |{q}_{s,m}| \cos \varphi_{s,m} - |{q}_{m,b}| \cos \varphi_{m,b} ) \bigr\} \qquad (16) \; (\leftarrow 1.5 式(v1)) \\
|{Q}_{i}| \sin \varphi_{i} = \sum_{m=1}^{M} & A_m \:\bigl\{ |{q}_{m,a}| \sin \varphi_{m,a} - |{q}_{s,m}| \sin \varphi_{s,m}  \\
&\hspace{12pt} + \delta_{wtype \neq 2} ( |{q}_{s,m}| \sin \varphi_{s,m} - |{q}_{m,b}| \sin \varphi_{m,b} ) \bigr\} \qquad (17) \; (\leftarrow 1.5 式(v2)) \\
\end{align} $$


#### (7) 有効熱容量

1.5 (7)をあわせて参照のこと

- 室内側吸熱有効熱容量${C}_{Ai}$の算定式：

$$C_{Ai} = \frac{1}{\omega} \: \sqrt{ |{Q}_{Ai}|^2 \cos^2 \varphi_{Ai} + |{Q}_{Ai}|^2 \sin^2 \varphi_{Ai} } = |{Q}_{Ai}| \big/ \omega  \qquad (18) \; (\leftarrow 1.5 式(w)) \\ $$ 

- 貫流有効熱容量${C}_{T}$の算定式：

$$C_{T} = \frac{1}{\omega} \: \sqrt{ |{Q}_{T}|^2 \cos^2 \varphi_{T} + |{Q}_{T}|^2 \sin^2 \varphi_{T} } = |{Q}_{T}| \big/ \omega  \qquad (19) \; (\leftarrow 1.5 式(x)) \\ $$ 

- 有効熱容量${C}_{i}$の算定式：

$$C_{i} = \frac{1}{\omega} \: \sqrt{ |{Q}_{i}|^2 \cos^2 \varphi_{i} + |{Q}_{i}|^2 \sin^2 \varphi_{i} } = |{Q}_{i}| \big/ \omega  \qquad (20) \; (\leftarrow 1.5 式(y)) \\ $$

### 1.3 計算プログラム

#### (1) 一次元壁体構成から周波数応答を経て有効熱容量を計算するプログラム本体

In [1]:
""" 一次元壁体構成から周波数応答を経て有効熱容量を計算するプログラム本体 """
import cmath
import numpy as np

def calc_specific_heat_capacity(AAA, Wtype, HHH, T_psi, NNN, d_ns, lambda_ns, c_rho_ns, TT):

    omega = [0 for t in range(len(TT))]
    CCAI = [0 for t in range(len(TT))]
    CCT = [0 for t in range(len(TT))]    
    CCI = [0 for t in range(len(TT))]   
    CC = 0.0
    CCm = [0 for i2 in range(len(AAA))]
    UUm = [0 for i2 in range(len(AAA))]
    UUA0 = 0.0
    UUA = 0.0
    CCsta = 0.0
    CCsta_m = [0 for i2 in range(len(AAA))]
    
    for T0 in TT:
        QCm = [[[[0 for i4 in range(3)] for i3 in range(2)] for i2 in range(len(AAA))] for i1 in range(len(TT))] 
        QSm = [[[[0 for i4 in range(3)] for i3 in range(2)] for i2 in range(len(AAA))] for i1 in range(len(TT))]           
        QAIC = [0 for t in range(len(TT))]   
        QAIS = [0 for t in range(len(TT))]   
        QTC = [0 for t in range(len(TT))]   
        QTS = [0 for t in range(len(TT))]         
        QIC = [0 for t in range(len(TT))]   
        QIS = [0 for t in range(len(TT))]         
        
        t = TT.index(T0)
        omega[t] = 2 * cmath.pi / (T0 * 3600)         # 周期Tと角周波数ω (1.2 (2))

        NStart = 0
        Atotal = 0.0      # Atotal: 躯体の総面積[m2]の初期化
        Atotal1 = 0.0     # Atotal1: UA値を計算するための外皮の総面積[m2]の初期化
        for AA in AAA:                                # AA: 躯体mの表面積[m2]
            m = AAA.index(AA)                         # m: 躯体構成の番号
            Atotal += AA                              # AA: 躯体m=1～Mの総表面積(片側)[m2]
            if Wtype[m]==0:
                Atotal1 += AA                         # Atotal1: UA値を計算するための外皮の総面積[m2]
            NN = NNN[m]                               # NN: 躯体mの層数N            
            d_n = d_ns[NStart:NStart+NN]              # d_nに抽出→躯体mの各層の厚さd_n[m]
            lambda_n = lambda_ns[NStart:NStart+NN]    # lambda_nに抽出→躯体mの各層の熱伝導率λ_n[W/(mK)]
            c_rho_n =  c_rho_ns[NStart:NStart+NN]     # c_rho_nに抽出→躯体mの容積熱容量cρ_n[kJ/(m3K)]
            NStart += NN                              # 次の躯体m+1のためにNの開始位置を更新
            
            if t==0:
                """ 熱容量の積算 (1.2 (1))"""
                for n in range(len(d_n)):
                    CCm[m] += c_rho_n[n] * d_n[n]           # 躯体mの熱容量C_m[kJ/(m2K)] (1.2 (1))
                    UUm[m] += d_n[n] / lambda_n[n]          # ここでは熱抵抗の積算
                CC += CCm[m] * AA                           # 躯体mの熱容量積算C[kJ/K] (1.2 (1))
                UUm[m] = 1 / UUm[m]                         # 躯体mの熱貫流率Um[W/(m2K)]
                UUA0 += UUm[m] * AA * HHH[m]                # 熱貫流率の積算[W/K](温度差係数考慮) (1.2 (1))
                for n in range(len(d_n)):
                    CCsta_ms = d_n[n] / ( 2 * lambda_n[n] ) 
                    for nn in range(n):
                        CCsta_ms += d_n[nn] / lambda_n[nn]
                    CCsta_m[m] += c_rho_n[n] * d_n[n] * (HHH[m] - UUm[m] * HHH[m] * CCsta_ms )
                CCsta_m[m] *= AA  
                CCsta += CCsta_m[m]                         # 内外1℃差(温度差係数考慮)、定常状態での蓄熱量[kJ/K]


            """ 各躯体構成で四端子行列を算定 (1.2 (3), (4)) """
            ZZ = calc_zm(d_n, lambda_n, c_rho_n, omega[t])  # 躯体構成の四端子行列算定モジュール呼び出し
            
            """ 熱流の周波数応答の係数の算定 (1.2 (5)) """
            # QCm[周期T][躯体m][励振の側][応答の側]：周期T、躯体mの「励振の側」励振における「応答の側」の周波数応答の係数|q|cosφ
            # QSm[周期T][躯体m][励振の側][応答の側]：周期T、躯体mの「励振の側」励振における「応答の側」の周波数応答の係数|q|sinφ
                # 「励振の側」「応答の側」：インデックスが「0」→a側、「1」→b側
                # 「励振の側」が0、「応答の側」が2 → 定常成分
            rr_aa = abs(ZZ[0,0]) / abs(ZZ[0,1])             # a側励振時a側熱流応答の振幅(1.2 式(8)第1,2式)
            pp_aa = cmath.phase(ZZ[0,0]) - cmath.phase(ZZ[0,1]) + cmath.pi       
                                                            # a側励振時a側熱流応答の位相差(1.2 式(8)第1,2式)
            QCm[t][m][0][0] = rr_aa * np.cos(pp_aa)         # a側励振時a側熱流応答の|q|cosφ(1.2 式(8)第1式)
            QSm[t][m][0][0] = rr_aa * np.sin(pp_aa)         # a側励振時a側熱流応答の|q|sinφ(1.2 式(8)第2式)
            
            rr_ab = 1 / abs(ZZ[0,1])                        # a側励振時b側熱流応答の振幅(1.2 式(8)第3,4式)
            pp_ab = - cmath.phase(ZZ[0,1]) - cmath.pi       # a側励振時b側熱流応答の位相差(1.2 式(8)第3,4式)
            QCm[t][m][0][1] = rr_ab * np.cos(pp_ab)         # a側励振時b側熱流応答の|q|cosφ(1.2 式(8)第3式)
            QSm[t][m][0][1] = rr_ab * np.sin(pp_ab)         # a側励振時b側熱流応答の|q|sinφ(1.2 式(8)第4式)
            
            psi_m = 2 * cmath.pi * T_psi[m] / T0
            rr_ba = ( 1 - HHH[m] ) / abs(ZZ[0,1])           # b側励振時a側熱流応答の振幅(1.2 式(9)第1,2式)
            pp_ba = psi_m - cmath.phase(ZZ[0,1])            # b側励振時a側熱流応答の位相差(1.2 式(9)第1,2式) 
            QCm[t][m][1][0] = rr_ba * np.cos(pp_ba)         # b側励振時a側熱流応答の|q|cosφ(1.2 式(9)第1式)
            QSm[t][m][1][0] = rr_ba * np.sin(pp_ba)         # b側励振時a側熱流応答の|q|sinφ(1.2 式(9)第2式)            

            rr_bb = ( 1 - HHH[m] ) * abs(ZZ[1,1]) / abs(ZZ[0,1])
                                                            # b側励振時b側熱流応答の振幅(1.2 式(9)第3,4式)
            pp_bb = psi_m - cmath.phase(ZZ[0,1]) + cmath.phase(ZZ[1,1])  
                                                            # b側励振時b側熱流応答の位相差(1.2 式(9)第3,4式)
            QCm[t][m][1][1] = rr_bb * np.cos(pp_bb)         # b側励振時b側熱流応答の|q|cosφ(1.2 式(9)第3式)
            QSm[t][m][1][1] = rr_bb * np.sin(pp_bb)         # b側励振時b側熱流応答の|q|sinφ(1.2 式(9)第4式)
            
            QCm[t][m][0][2] = UUm[m] * ( 1 - np.cos(psi_m) * ( 1 - HHH[m] ) ) #熱流定常成分のcosωtの係数(1.2 式(11)第1式)           
            QSm[t][m][0][2] = UUm[m] * np.sin(psi_m) * ( 1 - HHH[m] )         #熱流定常成分のsinωtの係数(1.2 式(11)第2式) 

            #print (QCm,QSm)
            
            """ 有効熱容量算定のために表面積を熱流にかけて積算 """ 
            QAIC[t] += AA * ( QCm[t][m][0][0] + QCm[t][m][1][0] - QCm[t][m][0][2] )
            QAIS[t] += AA * ( QSm[t][m][0][0] + QSm[t][m][1][0] - QSm[t][m][0][2] )
            if Wtype[m]==0:
                QTC[t] += AA * ( QCm[t][m][0][2] - QCm[t][m][0][1] - QCm[t][m][1][1] )
                QTS[t] += AA * ( QSm[t][m][0][2] - QSm[t][m][0][1] - QSm[t][m][1][1] )
                QIC[t] += AA * ( QCm[t][m][0][0] + QCm[t][m][1][0] - QCm[t][m][0][1] - QCm[t][m][1][1] )
                QIS[t] += AA * ( QSm[t][m][0][0] + QSm[t][m][1][0] - QSm[t][m][0][1] - QSm[t][m][1][1] )
            elif Wtype[m]==1:
                QAIC[t] += AA * ( QCm[t][m][0][2] - QCm[t][m][0][1] - QCm[t][m][1][1] )
                QAIS[t] += AA * ( QSm[t][m][0][2] - QSm[t][m][0][1] - QSm[t][m][1][1] )
                QIC[t] += AA * ( QCm[t][m][0][0] + QCm[t][m][1][0] - QCm[t][m][0][1] - QCm[t][m][1][1] )
                QIS[t] += AA * ( QSm[t][m][0][0] + QSm[t][m][1][0] - QSm[t][m][0][1] - QSm[t][m][1][1] )
            elif Wtype[m]==2:
                QIC[t] += AA * ( QCm[t][m][0][0] + QCm[t][m][1][0] - QCm[t][m][0][2] )
                QIS[t] += AA * ( QSm[t][m][0][0] + QSm[t][m][1][0] - QSm[t][m][0][2] )           

        """ 複数躯体で積算した熱流の周波数応答から有効熱容量を算定 (1.2 (7)) """ 
        CCAI[t] = 1 / omega[t] * ( QAIC[t]**2 + QAIS[t]**2 )**0.5 / 1000   
        CCT[t] = 1 / omega[t] * ( QTC[t]**2 + QTS[t]**2 )**0.5 / 1000
        CCI[t] = 1 / omega[t] * ( QIC[t]**2 + QIS[t]**2 )**0.5 / 1000
        
        if Atotal1 > 0:
            UUA = UUA0 / Atotal1                 # UA値[W/(m2K)]の計算 (1.2 (1))
        
        
    return [Atotal,Atotal1,CC,CCsta,UUA,TT,CCAI,CCT,CCI,QAIC,QAIS,QTC,QTS]


#### (2) 行列作成モジュール `calc_zm`

In [2]:
""" 層1～層Nの四端子行列作成 --- 1.2 (4) に対応した計算モジュール """
import cmath
import numpy as np

def calc_zm(d_n, lambda_n, c_rho_n, omega):
    ZZ = np.array([[1,0], [0,1]])
    for n in range(len(d_n)-1,-1,-1):
        ZZ = np.dot(ZZ, calc_zmn(d_n[n], lambda_n[n],c_rho_n[n], omega)) # 2×2の行列同士の積  1.2 式(6)
    return ZZ

#### (3) 各層の行列作成モジュール `calc_zmn`

In [3]:
""" 各層の四端子行列作成 --- 1.2 (3) に対応した計算モジュール """
import cmath
import numpy as np

def calc_zmn(d, lamb, c_rho, omega):
    if c_rho > 0:
        c_rho *= 1000  #面積熱容量cρは[kJ]だてで受取→[J]だてに変換
        delta = ( (2 * lamb) / (omega * c_rho) )**0.5       #1.2 式(3)
        xi = d / delta                                      #1.2 式(3)
        Z_11 = np.cosh(xi) * np.cos(xi) + (np.sinh(xi) * np.sin(xi))*1j
        Z_12 = - delta / (2 * lamb) * (( np.sinh(xi) * np.cos(xi) + np.cosh(xi) * np.sin(xi) ) 
                                     + ( np.cosh(xi) * np.sin(xi) - np.sinh(xi) * np.cos(xi) )*1j)
        Z_21 = - lamb / delta * (( np.sinh(xi) * np.cos(xi) - np.cosh(xi) * np.sin(xi) ) 
                               + ( np.sinh(xi) * np.cos(xi) + np.cosh(xi) * np.sin(xi) )*1j)    #1.2 式(4)
    else:
        Z_11 = 1
        Z_12 = - d / lamb
        Z_21 = 0               #1.2 式(5)
    
    Z_n = np.array([[Z_11,Z_12], [Z_21,Z_11]])
    return Z_n


### 1.4 計算例



In [4]:
""" 一次元壁体構成から周波数応答計算を経て有効熱容量を算定 """
""" 計算その1, 「高断熱・大熱容量床」単体 """ 

AAA = [1]                                                       # 複数躯体構成｢1組｣のそれぞれの表面積[m2]
HHH = [1.0]                                                     # 複数躯体構成を介して隣接する空間に適用する温度差係数[-]→外気：Hm = 1
T_psi = [0.0]                                                   # 複数躯体構成を介して隣接する空間との温度変動の位相のずれ[hour]
                                                                  # →原則ずれはなし：T_psi = 0
NNN = [6]                                                       # 複数躯体構成｢1組｣のそれぞれの層数N
d_ns =      [1,     0.13448,  0.012,   0.1,    0.045,  1]       # 1組目の各層の厚さd[m]
lambda_ns = [6.7,   1.6,      0.16,    0.038,  0.028,  6.7]     # 1組目の各層の熱伝導率λ[W/(mK)]
c_rho_ns =  [0,     1896.260, 715.806, 13.395, 25.116, 0]       # 1組目の各層の容積比熱cρ[kJ/(m3K)] 
TT = [24]                                                       # 計算する周期T[hour]
Wtype = [0]                                                     # 壁のタイプ:0 → 外壁扱い

[Atotal,Atotal1,CC,CCsta,UUA,TT,CCAI,CCT,CCI,QAIC,QAIS,QTC,QTS] \
    = calc_specific_heat_capacity(AAA, Wtype, HHH, T_psi, NNN, d_ns, lambda_ns, c_rho_ns, TT)

print (Atotal,Atotal1,CC,CCsta,UUA,TT,CCAI,CCT,CCI,QAIC,QAIS,QTC,QTS)
#print ('(|q|cosφ)a,a = {} [W/(m2K)], (|q|sinφ)a,a = {} [/(m2K)]'.format( QC[0][0][0], QS[0][0][0] ))
#print ('(|q|cosφ)a,b = {} [W/(m2K)], (|q|sinφ)a,b = {} [W/(m2K)]'.format( QC[0][0][1], QS[0][0][1] ))
print ('T = {} [hour] におけるa側有効熱容量 Ci = {} [kJ/(m2K)]'.format( TT[0], CCI[0] ))
print ('T = {} [hour] におけるa側有効吸熱熱容量 CAi = {} [kJ/(m2K)]'.format( TT[0], CCAI[0] ))
print ('T = {} [hour] における有効貫流熱容量 CT = {} [kJ/(m2K)]'.format( TT[0], CCT[0] ))
print ('C = {} [kJ/(m2K)],  Cstastic = {} [kJ/(m2K)]'.format( CC, CCsta ))

# セルの参照先は、\周波数応答-単位応答変換(高断熱大熱容量床).xlsm 内の「周期定常→指数項二項近似」シート内の出力値
# (|q|cosφ)a,a = 5.112460236,  (|q|sinφ)a,a = 1.591950776      セルR23C6,R23C7
# (|q|cosφ)a,b = -0.007437852,  (|q|sinφ)a,b = -0.060418385    セルR23C10,R23C11
# Cs = 73.97939071                                               セルR23C26
# C = 266.0684568                                                セルR4C10
# Cstastic = 253.8225169                                         セルR4C12


1.0 1.0 266.0684368 253.8225037802193 0.21293452605868463 [24] [70.840488737566702] [3.142163309465571] [73.979390097265977] [4.8995257138759207] [1.5919508295206453] [0.22037230468600044] [0.06041840071398695]
T = 24 [hour] におけるa側有効熱容量 Ci = 73.97939009726598 [kJ/(m2K)]
T = 24 [hour] におけるa側有効吸熱熱容量 CAi = 70.8404887375667 [kJ/(m2K)]
T = 24 [hour] における有効貫流熱容量 CT = 3.142163309465571 [kJ/(m2K)]
C = 266.0684368 [kJ/(m2K)],  Cstastic = 253.8225037802193 [kJ/(m2K)]


In [5]:
""" 一次元壁体構成から周波数応答計算を経て有効熱容量を算定 """
""" 計算その2, 「高断熱・小熱容量床」単体 """ 

AAA = [1]                                                       # 複数躯体構成｢1組｣のそれぞれの表面積[m2]
HHH = [1.0]                                                     # 複数躯体構成を介して隣接する空間に適用する温度差係数[-]→外気：Hm = 1
T_psi = [0.0]                                                   # 複数躯体構成を介して隣接する空間との温度変動の位相のずれ[hour]
                                                                  # →原則ずれはなし：T_psi = 0
NNN = [5]                                                       # 複数躯体構成｢1組｣のそれぞれの層数N
d_ns =      [1,     0.012,   0.1,    0.045,  1]                 # 1組目の各層の厚さd[m]
lambda_ns = [6.7,   0.16,    0.038,  0.028,  6.7]               # 1組目の各層の熱伝導率λ[W/(mK)]
c_rho_ns =  [0,     715.806, 13.395, 25.116, 0]                 # 1組目の各層の容積比熱cρ[kJ/(m3K)] 
TT = [24]                                                       # 計算する周期T[hour]
Wtype = [0]                                                     # 壁のタイプ:0 → 外壁扱い

[Atotal,Atotal1,CC,CCsta,UUA,TT,CCAI,CCT,CCI,QAIC,QAIS,QTC,QTS] \
    = calc_specific_heat_capacity(AAA, Wtype, HHH, T_psi, NNN, d_ns, lambda_ns, c_rho_ns, TT)

print (Atotal,Atotal1,CC,CCsta,UUA,TT,CCAI,CCT,CCI,QAIC,QAIS,QTC,QTS)
# print ('(|q|cosφ)a,a = {} [W/(m2K)], (|q|sinφ)a,a = {} [W/(m2K)]'.format( QC[0][0][0], QS[0][0][0] ))
# print ('(|q|cosφ)a,b = {} [W/(m2K)], (|q|sinφ)a,b = {} [W/(m2K)]'.format( QC[0][0][1], QS[0][0][1] ))
print ('T = {} [hour] におけるa側有効熱容量 Ci = {} [kJ/(m2K)]'.format( TT[0], CCI[0] ))
print ('T = {} [hour] におけるa側有効吸熱熱容量 CAi = {} [kJ/(m2K)]'.format( TT[0], CCAI[0] ))
print ('T = {} [hour] における有効貫流熱容量 CT = {} [kJ/(m2K)]'.format( TT[0], CCT[0] ))
print ('C = {} [kJ/(m2K)],  Cstastic = {} [kJ/(m2K)]'.format( CC, CCsta ))

# セルの参照先は、\周波数応答-単位応答変換(高断熱小熱容量床).xlsm 内の「周期定常→指数項二項近似」シート内の出力値
# (|q|cosφ)a,a = 0.289974068,  (|q|sinφ)a,a = 0.616355963      セルR23C6,R23C7
# (|q|cosφ)a,b = 0.207441047,  (|q|sinφ)a,b = -0.05454317      セルR23C10,R23C11
# Cs = 9.295070697                                               セルR23C26
# C = 11.059412                                                  セルR4C10
# Cstastic = 9.367604583                                         セルR4C12


1.0 1.0 11.059392 9.367591261520344 0.21681489407128907 [24] [8.534989939853066] [0.76101444168950094] [9.2950579118645305] [0.073158952508141084] [0.61635532973325291] [0.0093737571235967543] [0.054542905291009858]
T = 24 [hour] におけるa側有効熱容量 Ci = 9.29505791186453 [kJ/(m2K)]
T = 24 [hour] におけるa側有効吸熱熱容量 CAi = 8.534989939853066 [kJ/(m2K)]
T = 24 [hour] における有効貫流熱容量 CT = 0.7610144416895009 [kJ/(m2K)]
C = 11.059392 [kJ/(m2K)],  Cstastic = 9.367591261520344 [kJ/(m2K)]


In [6]:
""" 一次元壁体構成から周波数応答計算を経て有効熱容量を算定 """
""" 計算その3, 「低断熱・大熱容量床」単体 """ 

AAA = [1]                                                       # 複数躯体構成｢1組｣のそれぞれの表面積[m2]
HHH = [1.0]                                                     # 複数躯体構成を介して隣接する空間に適用する温度差係数[-]→外気：Hm = 1
T_psi = [0.0]                                                   # 複数躯体構成を介して隣接する空間との温度変動の位相のずれ[hour]
                                                                  # →原則ずれはなし：T_psi = 0
NNN = [5]                                                       # 複数躯体構成｢1組｣のそれぞれの層数N
d_ns =      [1,     0.13448, 0.012,   0.02,   1]                # 1組目の各層の厚さd[m]
lambda_ns = [6.7,   1.6,     0.16,    0.038,  6.7]              # 1組目の各層の熱伝導率λ[W/(mK)]
c_rho_ns =  [0,     1896.26, 715.806, 56.511, 0]                # 1組目の各層の容積比熱cρ[kJ/(m3K)] 
TT = [24]                                                       # 計算する周期T[hour]
Wtype = [0]                                                     # 壁のタイプ:0 → 外壁扱い

[Atotal,Atotal1,CC,CCsta,UUA,TT,CCAI,CCT,CCI,QAIC,QAIS,QTC,QTS] \
    = calc_specific_heat_capacity(AAA, Wtype, HHH, T_psi, NNN, d_ns, lambda_ns, c_rho_ns, TT)

print (Atotal,Atotal1,CC,CCsta,UUA,TT,CCAI,CCT,CCI,QAIC,QAIS,QTC,QTS)
# print ('(|q|cosφ)a,a = {} [W/(m2K)], (|q|sinφ)a,a = {} [W/(m2K)]'.format( QC[0][0][0], QS[0][0][0] ))
# print ('(|q|cosφ)a,b = {} [W/(m2K)], (|q|sinφ)a,b = {} [W/(m2K)]'.format( QC[0][0][1], QS[0][0][1] ))
print ('T = {} [hour] におけるa側有効熱容量 Ci = {} [kJ/(m2K)]'.format( TT[0], CCI[0] ))
print ('T = {} [hour] におけるa側有効吸熱熱容量 CAi = {} [kJ/(m2K)]'.format( TT[0], CCAI[0] ))
print ('T = {} [hour] における有効貫流熱容量 CT = {} [kJ/(m2K)]'.format( TT[0], CCT[0] ))
print ('C = {} [kJ/(m2K)],  Cstastic = {} [kJ/(m2K)]'.format( CC, CCsta ))

# セルの参照先は、\周波数応答-単位応答変換(低断熱大熱容量床).xlsm 内の「周期定常→指数項二項近似」シート内の出力値
# (|q|cosφ)a,a = 5.033717243,  (|q|sinφ)a,a = 1.586148084      セルR23C6,R23C7
# (|q|cosφ)a,b = 0.023702745,  (|q|sinφ)a,b = -0.348024998     セルR23C10,R23C11
# Cs = 73.84839656                                               セルR23C26
# C = 264.7289368                                                セルR4C10
# Cstastic = 212.1309024                                         セルR4C12


1.0 1.0 264.7289368 212.13090240438572 1.0163910826972273 [24] [59.392127748129141] [14.465041917473155] [73.848396560847775] [4.0173261602499526] [1.5861480838969899] [0.99268833758826303] [0.34802499754292981]
T = 24 [hour] におけるa側有効熱容量 Ci = 73.84839656084777 [kJ/(m2K)]
T = 24 [hour] におけるa側有効吸熱熱容量 CAi = 59.39212774812914 [kJ/(m2K)]
T = 24 [hour] における有効貫流熱容量 CT = 14.465041917473155 [kJ/(m2K)]
C = 264.7289368 [kJ/(m2K)],  Cstastic = 212.13090240438572 [kJ/(m2K)]


In [7]:
""" 一次元壁体構成から周波数応答計算を経て有効熱容量を算定 """
""" 計算その4, 「低断熱・小熱容量床」単体 """ 

AAA = [1]                                                       # 複数躯体構成｢1組｣のそれぞれの表面積[m2]
HHH = [1.0]                                                     # 複数躯体構成を介して隣接する空間に適用する温度差係数[-]→外気：Hm = 1
T_psi = [0.0]                                                   # 複数躯体構成を介して隣接する空間との温度変動の位相のずれ[hour]
                                                                  # →原則ずれはなし：T_psi = 0
NNN = [4]                                                       # 複数躯体構成｢1組｣のそれぞれの層数N
d_ns =      [1,     0.012,   0.02,   1]                         # 1組目の各層の厚さd[m]
lambda_ns = [6.7,   0.16,    0.038,  6.7]                       # 1組目の各層の熱伝導率λ[W/(mK)]
c_rho_ns =  [0,     715.806, 56.511, 0]                         # 1組目の各層の容積比熱cρ[kJ/(m3K)] 
TT = [24]                                                       # 計算する周期T[hour]
Wtype = [0]                                                     # 壁のタイプ:0 → 外壁扱い

[Atotal,Atotal1,CC,CCsta,UUA,TT,CCAI,CCT,CCI,QAIC,QAIS,QTC,QTS] \
    = calc_specific_heat_capacity(AAA, Wtype, HHH, T_psi, NNN, d_ns, lambda_ns, c_rho_ns, TT)

print (Atotal,Atotal1,CC,CCsta,UUA,TT,CCAI,CCT,CCI,QAIC,QAIS,QTC,QTS)
# print ('(|q|cosφ)a,a = {} [W/(m2K)], (|q|sinφ)a,a = {} [W/(m2K)]'.format( QC[0][0][0], QS[0][0][0] ))
# print ('(|q|cosφ)a,b = {} [W/(m2K)], (|q|sinφ)a,b = {} [W/(m2K)]'.format( QC[0][0][1], QS[0][0][1] ))
print ('T = {} [hour] におけるa側有効熱容量 Ci = {} [kJ/(m2K)]'.format( TT[0], CCI[0] ))
print ('T = {} [hour] におけるa側有効吸熱熱容量 CAi = {} [kJ/(m2K)]'.format( TT[0], CCAI[0] ))
print ('T = {} [hour] における有効貫流熱容量 CT = {} [kJ/(m2K)]'.format( TT[0], CCT[0] ))
print ('C = {} [kJ/(m2K)],  Cstastic = {} [kJ/(m2K)]'.format( CC, CCsta ))

# セルの参照先は、\周波数応答-単位応答変換(低断熱大熱容量床).xlsm 内の「周期定常→指数項二項近似」シート内の出力値
# (|q|cosφ)a,a = 1.14795671,  (|q|sinφ)a,a = 0.408941623       セルR23C6,R23C7
# (|q|cosφ)a,b = 1.100322652,  (|q|sinφ)a,b = -0.119442268     セルR23C10,R23C11
# Cs = 7.29526529                                                セルR23C26
# C = 9.719892                                                   セルR4C10
# Cstastic = 7.324937591                                         セルR4C12

1.0 1.0 9.719892 7.3249375910430174 1.1113293611820423 [24] [5.6458614778031775] [1.6494080062861118] [7.2952652898412023] [0.036627348955247108] [0.40894162311192739] [0.011006708916821317] [0.11944226825765084]
T = 24 [hour] におけるa側有効熱容量 Ci = 7.295265289841202 [kJ/(m2K)]
T = 24 [hour] におけるa側有効吸熱熱容量 CAi = 5.6458614778031775 [kJ/(m2K)]
T = 24 [hour] における有効貫流熱容量 CT = 1.6494080062861118 [kJ/(m2K)]
C = 9.719892 [kJ/(m2K)],  Cstastic = 7.3249375910430174 [kJ/(m2K)]


In [8]:
""" 一次元壁体構成から周波数応答計算を経て有効熱容量を算定 """
""" 計算その5, ISO 13786: 2007 Annex D  D.2の例 """ 

AAA = [1]                                                       # 複数躯体構成｢1組｣のそれぞれの表面積[m2]
HHH = [1.0]                                                     # 複数躯体構成を介して隣接する空間に適用する温度差係数[-]→外気：Hm = 1
T_psi = [0.0]                                                   # 複数躯体構成を介して隣接する空間との温度変動の位相のずれ[hour]
                                                                  # →原則ずれはなし：T_psi = 0
NNN = [5]                                                       # 複数躯体構成｢1組｣のそれぞれの層数N
d_ns =      [1,      0.20,    0.10,   0.005,  1]                # 1組目の各層の厚さd[m]
lambda_ns = [1/0.13, 1.80,    0.04,   1.0,    1/0.040]          # 1組目の各層の熱伝導率λ[W/(mK)]
c_rho_ns =  [0,      2400.0,  42.0,   1800.0, 0]                # 1組目の各層の容積比熱cρ[kJ/(m3K)] 
TT = [24]                                                       # 計算する周期T[hour]
Wtype = [0]                                                     # 壁のタイプ:0 → 外壁扱い

[Atotal,Atotal1,CC,CCsta,UUA,TT,CCAI,CCT,CCI,QAIC,QAIS,QTC,QTS] \
    = calc_specific_heat_capacity(AAA, Wtype, HHH, T_psi, NNN, d_ns, lambda_ns, c_rho_ns, TT)

print (Atotal,Atotal1,CC,CCsta,UUA,TT,CCAI,CCT,CCI,QAIC,QAIS,QTC,QTS)
# print ('(|q|cosφ)a,a = {} [W/(m2K)], (|q|sinφ)a,a = {} [W/(m2K)]'.format( QC[0][0][0], QS[0][0][0] ))
# print ('(|q|cosφ)a,b = {} [W/(m2K)], (|q|sinφ)a,b = {} [W/(m2K)]'.format( QC[0][0][1], QS[0][0][1] ))
print ('T = {} [hour] におけるa側有効熱容量 Ci = {} [kJ/(m2K)]'.format( TT[0], CCI[0] ))
print ('T = {} [hour] におけるa側有効吸熱熱容量 CAi = {} [kJ/(m2K)]'.format( TT[0], CCAI[0] ))
print ('T = {} [hour] における有効貫流熱容量 CT = {} [kJ/(m2K)]'.format( TT[0], CCT[0] ))
print ('C = {} [kJ/(m2K)],  Cstastic = {} [kJ/(m2K)]'.format( CC, CCsta ))

# ISO 13786: 2007  p.21  Annex D  D.2 Table D.4
# Internal areal heat capacity, κ1 = 82[kJ/(m2K)]


1.0 1.0 493.2 450.1213758723828 0.3589232303090728 [24] [76.899841283751329] [5.4188726870963899] [82.290128152755969] [5.435291792110994] [1.3158959011448188] [0.39068381311602052] [0.051561017911026803]
T = 24 [hour] におけるa側有効熱容量 Ci = 82.29012815275597 [kJ/(m2K)]
T = 24 [hour] におけるa側有効吸熱熱容量 CAi = 76.89984128375133 [kJ/(m2K)]
T = 24 [hour] における有効貫流熱容量 CT = 5.41887268709639 [kJ/(m2K)]
C = 493.2 [kJ/(m2K)],  Cstastic = 450.1213758723828 [kJ/(m2K)]


In [9]:
""" 一次元壁体構成から周波数応答計算を経て有効熱容量を算定 """
""" 計算その6, 「間仕切壁」単体 """ 

AAA = [1]                                                       # 複数躯体構成｢1組｣のそれぞれの表面積[m2]
HHH = [0.0]                                                     # 複数躯体構成を介して隣接する空間に適用する温度差係数[-]→外気：Hm = 1
T_psi = [0.0]                                                   # 複数躯体構成を介して隣接する空間との温度変動の位相のずれ[hour]
                                                                  # →原則ずれはなし：T_psi = 0
NNN = [5]                                                       # 複数躯体構成｢1組｣のそれぞれの層数N
d_ns =      [1,      0.0125,  0.10,     0.0125,  1]             # 1組目の各層の厚さd[m]
lambda_ns = [1/0.11, 0.22,    0.1/0.09, 0.22,    1/0.11]        # 1組目の各層の熱伝導率λ[W/(mK)]
c_rho_ns =  [0,      904.176, 1.298,    904.176, 0]             # 1組目の各層の容積比熱cρ[kJ/(m3K)] 
TT = [24]                                                       # 計算する周期T[hour]
Wtype = [1]                                                     # 壁のタイプ:1 → 内壁扱い

[Atotal,Atotal1,CC,CCsta,UUA,TT,CCAI,CCT,CCI,QAIC,QAIS,QTC,QTS] \
    = calc_specific_heat_capacity(AAA, Wtype, HHH, T_psi, NNN, d_ns, lambda_ns, c_rho_ns, TT)

print (Atotal,Atotal1,CC,CCsta,UUA,TT,CCAI,CCT,CCI,QAIC,QAIS,QTC,QTS)
print ('T = {} [hour] におけるa側有効熱容量 Ci = {} [kJ/(m2K)]'.format( TT[0], CCI[0] ))
print ('T = {} [hour] におけるa側有効吸熱熱容量 CAi = {} [kJ/(m2K)]'.format( TT[0], CCAI[0] ))
print ('T = {} [hour] における有効貫流熱容量 CT = {} [kJ/(m2K)]'.format( TT[0], CCT[0] ))
print ('C = {} [kJ/(m2K)],  Cstastic = {} [kJ/(m2K)]'.format( CC, CCsta ))

# セルの参照先は、\周波数応答-単位応答変換(間仕切壁検討).xlsm 内の「有効熱容量確認」シート内の出力値
# Ci = 22.60482934                                               セルR26C16
# CAi = 11.30241467 + 11.30241467                                セルR26C19とセルR26C20の和
# C = 22.7342                                                    セルR4C10

1.0 0.0 22.7342 0.0 0.0 [24] [22.604829343918478] [0.0] [22.604829343918482] [0.17436416116407694] [1.6345960758792075] [0] [0]
T = 24 [hour] におけるa側有効熱容量 Ci = 22.60482934391848 [kJ/(m2K)]
T = 24 [hour] におけるa側有効吸熱熱容量 CAi = 22.604829343918478 [kJ/(m2K)]
T = 24 [hour] における有効貫流熱容量 CT = 0.0 [kJ/(m2K)]
C = 22.7342 [kJ/(m2K)],  Cstastic = 0.0 [kJ/(m2K)]


In [10]:
""" 一次元壁体構成から周波数応答計算を経て有効熱容量を算定 """
""" 計算その7, 「階間床」単体 """ 

AAA = [1]                                                       # 複数躯体構成｢1組｣のそれぞれの表面積[m2]
HHH = [0.0]                                                     # 複数躯体構成を介して隣接する空間に適用する温度差係数[-]→外気：Hm = 1
T_psi = [0.0]                                                   # 複数躯体構成を介して隣接する空間との温度変動の位相のずれ[hour]
                                                                  # →原則ずれはなし：T_psi = 0
NNN = [4]                                                       # 複数躯体構成｢1組｣のそれぞれの層数N
d_ns =      [1,      0.09,    0.012,    1]                      # 1組目の各層の厚さd[m]
lambda_ns = [6.7,    1.60,    0.160,    6.7]                    # 1組目の各層の熱伝導率λ[W/(mK)]
c_rho_ns =  [0,      1896.260,715.806,  0]                      # 1組目の各層の容積比熱cρ[kJ/(m3K)] 
TT = [24]                                                       # 計算する周期T[hour]
Wtype = [1]                                                     # 壁のタイプ:1 → 内壁扱い

[Atotal,Atotal1,CC,CCsta,UUA,TT,CCAI,CCT,CCI,QAIC,QAIS,QTC,QTS] \
    = calc_specific_heat_capacity(AAA, Wtype, HHH, T_psi, NNN, d_ns, lambda_ns, c_rho_ns, TT)

print (Atotal,Atotal1,CC,CCsta,UUA,TT,CCAI,CCT,CCI,QAIC,QAIS,QTC,QTS)
print ('T = {} [hour] におけるa側有効熱容量 Ci = {} [kJ/(m2K)]'.format( TT[0], CCI[0] ))
print ('T = {} [hour] におけるa側有効吸熱熱容量 CAi = {} [kJ/(m2K)]'.format( TT[0], CCAI[0] ))
print ('T = {} [hour] における有効貫流熱容量 CT = {} [kJ/(m2K)]'.format( TT[0], CCT[0] ))
print ('C = {} [kJ/(m2K)],  Cstastic = {} [kJ/(m2K)]'.format( CC, CCsta ))

# セルの参照先は、\周波数応答-単位応答変換(間仕切壁検討).xlsm 内の「有効熱容量確認」シート内の出力値
# Ci = CAi    
# CAi = 65.97984161 + 47.79108244                                「有効熱容量確認」シートのセルR26C19と
#                                                                  「有効熱容量確認 (下階励振)」シートのセルR26C19の和
# C = 179.253072                                                 セルR4C10

1.0 0.0 179.253072 0.0 0.0 [24] [113.7707172082338] [0.0] [113.7707172082338] [6.3775195549791039] [5.2707080835641786] [0] [0]
T = 24 [hour] におけるa側有効熱容量 Ci = 113.7707172082338 [kJ/(m2K)]
T = 24 [hour] におけるa側有効吸熱熱容量 CAi = 113.7707172082338 [kJ/(m2K)]
T = 24 [hour] における有効貫流熱容量 CT = 0.0 [kJ/(m2K)]
C = 179.253072 [kJ/(m2K)],  Cstastic = 0.0 [kJ/(m2K)]


In [11]:
""" 一次元壁体構成から周波数応答計算を経て有効熱容量を算定 """
""" 計算その8, RCと断熱材の厚さに関するパラメトリックスタディ 構成1(室内側にせっこうボード＋中空層) """ 

AAA = [1]                                                       # 複数躯体構成｢1組｣のそれぞれの表面積[m2]
HHH = [1.0]                                                     # 複数躯体構成を介して隣接する空間に適用する温度差係数[-]→外気：Hm = 1
T_psi = [0.0]                                                   # 複数躯体構成を介して隣接する空間との温度変動の位相のずれ[hour]
                                                                  # →原則ずれはなし：T_psi = 0
TT = [24]                                                       # 計算する周期T[hour]
Wtype = [0]                                                     # 壁のタイプ:0 → 外壁扱い

d_rcs = [0.0, 0.025, 0.05, 0.075, 0.1, 0.125, 0.15, 0.175, 0.2, 0.225, 0.25]
# d_rcs = [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2]
d_ins = [-0.25, -0.225, -0.2, -0.175, -0.15, -0.125, -0.1, -0.075, -0.05, -0.025, 0, 0.025, 0.05, 0.075, 0.1, 0.125, 0.15, 0.175, 0.2, 0.225, 0.25]

for d_rc, d_in in [(d_rc, d_in) for d_rc in d_rcs for d_in in d_ins]:
    if d_rc == 0 and d_in==0:    # RC,断熱材なし
        NNN = [4]                                               # 複数躯体構成｢1組｣のそれぞれの層数N
        d_ns =      [1.0,   0.012,   0.010,  1.0]               # 1組目の各層の厚さd[m]
        lambda_ns = [6.7,   0.22,    0.111,  25.0]              # 1組目の各層の熱伝導率λ[W/(mK)]
        c_rho_ns =  [0.0,   830.0,   0.0,    0.0]               # 1組目の各層の容積比熱cρ[kJ/(m3K)] 
    elif d_in==0:    # 断熱材なし,RC厚可変
        NNN = [5]
        d_ns =      [1.0,   0.012,   0.010,  d_rc,    1.0]
        lambda_ns = [6.7,   0.22,    0.111,  1.6,     25.0]
        c_rho_ns =  [0.0,   830.0,   0.0,    2000.0,  0.0]      
    elif  d_rc == 0:    # RCなし,断熱材厚可変
        NNN = [5]
        d_ns =      [1.0,   0.012,   0.010,  abs(d_in),  1.0]
        lambda_ns = [6.7,   0.22,    0.111,  0.028,      25.0]
        c_rho_ns =  [0.0,   830.0,   0.0,    40.0,       0.0]
    elif  d_in > 0 :    # RC可変,外断熱
        NNN = [6]
        d_ns =      [1.0,   0.012,   0.010,  d_rc,    d_in,    1.0]
        lambda_ns = [6.7,   0.22,    0.111,  1.6,     0.028,   25.0]
        c_rho_ns =  [0.0,   830.0,   0.0,    2000.0,  40.0,    0.0]
    elif  d_in < 0 :    # RC可変,内断熱
        NNN = [6]
        d_ns =      [1.0,   0.012,   0.010,  abs(d_in),    d_rc,    1.0]
        lambda_ns = [6.7,   0.22,    0.111,  0.028,        1.6,     25.0]
        c_rho_ns =  [0.0,   830.0,   0.0,    40.0,         2000.0,  0.0]        

    [Atotal,Atotal1,CC,CCsta,UUA,TT,CCAI,CCT,CCI,QAIC,QAIS,QTC,QTS] \
        = calc_specific_heat_capacity(AAA, Wtype, HHH, T_psi, NNN, d_ns, lambda_ns, c_rho_ns, TT)
   
    print(d_rc,d_in,CC,CCsta,TT[0],CCAI[0],CCT[0],CCI[0])
    

0.0 -0.25 19.96 14.63312682835758 24 11.8227881466 1.51226330434 13.1492197999
0.0 -0.225 18.96 14.113423815317077 24 11.6895901595 1.49008087904 13.0535375833
0.0 -0.2 17.96 13.589015018518896 24 11.5014864937 1.44314062909 12.8663458988
0.0 -0.175 16.96 13.057985949824792 24 11.2533464701 1.37487034373 12.584603001
0.0 -0.15 15.96 12.517220676881028 24 10.9433595984 1.29211024009 12.2142188404
0.0 -0.125 14.96 11.961283983469182 24 10.5695514344 1.20504402869 11.7659025031
0.0 -0.1 13.96 11.379770126706223 24 10.1225590684 1.12813798166 11.2479120967
0.0 -0.075 12.96 10.749936815257794 24 9.57039756281 1.08489421583 10.6546762757
0.0 -0.05 11.96 10.01072210664382 24 8.81295315469 1.1259530469 9.93883258219
0.0 -0.025 10.96 8.923294339515047 24 7.46060553963 1.41247556947 8.87307770032
0.0 0 9.96 4.694171911747907 24 2.23131178913 2.45610175007 4.68741307653
0.0 0.025 10.96 8.923294339515047 24 7.46060553963 1.41247556947 8.87307770032
0.0 0.05 11.96 10.01072210664382 24 8.81295315469

0.125 -0.025 260.96 24.192800853420863 24 8.00170144491 10.1145200567 16.9586622432
0.125 0 259.96 53.66581924474352 24 9.68388628478 28.9987164393 38.5814267164
0.125 0.025 260.96 195.1952134231877 24 33.1406607341 10.9480862759 44.0640990724
0.125 0.05 261.96 222.13438555161463 24 37.4275347323 6.52843612315 43.9418538024
0.125 0.075 262.96 233.7972275971451 24 39.2331937118 4.67952959744 43.901934226
0.125 0.1 263.96 240.4559444993088 24 40.2265092666 3.66985040518 43.8866405222
0.125 0.125 264.96 244.85932256345586 24 40.8534560173 3.03585447385 43.8794175492
0.125 0.15 265.96 248.05449675815493 24 41.2839070068 2.60025357395 43.8731083382
0.125 0.175 266.96 250.52724514249266 24 41.5967991094 2.28012340196 43.8637814658
0.125 0.2 267.96 252.53370300026387 24 41.8339547052 2.03144178006 43.8492952083
0.125 0.225 268.96 254.22171983595405 24 42.019716238 1.82905216666 43.8290154877
0.125 0.25 269.96 255.68261401549984 24 42.1692494893 1.65816825301 43.8036690811
0.15 -0.25 319.96 17

0.25 -0.05 511.96 36.096352425789206 24 9.0224439881 7.03892433105 13.2594992475
0.25 -0.025 510.96 51.859584304336394 24 8.12834937306 11.5446573603 16.8686764243
0.25 0 509.96 126.87431248341707 24 14.7957368117 32.7382919694 47.1398497377
0.25 0.025 510.96 374.5446725305129 24 31.728554849 10.6500142812 42.2607394361
0.25 0.05 511.96 428.2765251224618 24 35.5251055433 6.42780566147 41.8856338568
0.25 0.075 512.96 452.0097793911309 24 37.1881988858 4.61251330781 41.7519755654
0.25 0.1 513.96 465.5282703935215 24 38.1210259354 3.60185899359 41.6831506289
0.25 0.125 514.96 474.35453063107207 24 38.717749161 2.95703876929 41.639884033
0.25 0.15 515.96 480.6380160350725 24 39.1322085401 2.50837220315 41.6082979232
0.25 0.175 516.96 485.38924044500857 24 39.4368583786 2.17634618148 41.5822808713
0.25 0.2 517.96 489.14590495421334 24 39.6703004827 1.91891418859 41.5589702613
0.25 0.225 518.96 492.22047091506926 24 39.8549745395 1.7120661317 41.5372198713
0.25 0.25 519.96 494.80691919612497

In [19]:
""" 一次元壁体構成から周波数応答計算を経て有効熱容量を算定 """
""" 計算その9, RCと断熱材の厚さに関するパラメトリックスタディ 構成2(RCと断熱材のみ) """ 

AAA = [1]                                                       # 複数躯体構成｢1組｣のそれぞれの表面積[m2]
HHH = [1.0]                                                     # 複数躯体構成を介して隣接する空間に適用する温度差係数[-]→外気：Hm = 1
T_psi = [0.0]                                                   # 複数躯体構成を介して隣接する空間との温度変動の位相のずれ[hour]
                                                                  # →原則ずれはなし：T_psi = 0
TT = [24]                                                       # 計算する周期T[hour]
Wtype = [0]                                                     # 壁のタイプ:0 → 外壁扱い

d_rcs = [0.0, 0.025, 0.05, 0.075, 0.1, 0.125, 0.15, 0.175, 0.2, 0.225, 0.25]
# d_rcs = [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2]
d_ins = [-0.25, -0.225, -0.2, -0.175, -0.15, -0.125, -0.1, -0.075, -0.05, -0.025, 0, 0.025, 0.05, 0.075, 0.1, 0.125, 0.15, 0.175, 0.2, 0.225, 0.25]

for d_rc, d_in in [(d_rc, d_in) for d_rc in d_rcs for d_in in d_ins]:
    if d_rc == 0 and d_in==0:    # RC,断熱材なし
        NNN = [2]                                               # 複数躯体構成｢1組｣のそれぞれの層数N
        d_ns =      [1.0,   1.0]               # 1組目の各層の厚さd[m]
        lambda_ns = [1/0.10989010989011,   25.0]              # 1組目の各層の熱伝導率λ[W/(mK)]
        c_rho_ns =  [0.0,   0.0]               # 1組目の各層の容積熱容量cρ[kJ/(m3K)] 
    elif d_in==0:    # 断熱材なし,RC厚可変
        NNN = [3]
        d_ns =      [1.0,   d_rc,    1.0]
        lambda_ns = [1/0.10989010989011,   1.6,     25.0]
        c_rho_ns =  [0.0,   2000.0,  0.0]      
    elif  d_rc == 0:    # RCなし,断熱材厚可変
        NNN = [3]
        d_ns =      [1.0,   abs(d_in),  1.0]
        lambda_ns = [1/0.10989010989011,   0.028,      25.0]
        c_rho_ns =  [0.0,   40.0,       0.0]
    elif  d_in > 0 :    # RC可変,外断熱
        NNN = [4]
        d_ns =      [1.0,   d_rc,    d_in,    1.0]
        lambda_ns = [1/0.10989010989011,   1.6,     0.028,   25.0]
        c_rho_ns =  [0.0,   2000.0,  40.0,    0.0]
    elif  d_in < 0 :    # RC可変,内断熱
        NNN = [4]
        d_ns =      [1.0,   abs(d_in),    d_rc,    1.0]
        lambda_ns = [1/0.10989010989011,   0.028,        1.6,     25.0]
        c_rho_ns =  [0.0,   40.0,         2000.0,  0.0]        

    [Atotal,Atotal1,CC,CCsta,UUA,TT,CCAI,CCT,CCI,QAIC,QAIS,QTC,QTS] \
        = calc_specific_heat_capacity(AAA, Wtype, HHH, T_psi, NNN, d_ns, lambda_ns, c_rho_ns, TT)
   
    print(d_rc,d_in,CC,CCsta,TT[0],CCAI[0],CCT[0],CCI[0])
    

0.0 -0.25 10.0 4.961507734766504 24 2.81183710895 1.3782485891 4.14628107622
0.0 -0.225 9.0 4.4615782196028935 24 2.63746400061 1.32218240526 3.93166250482
0.0 -0.2 8.0 3.9616659634741733 24 2.42260594283 1.23721298198 3.64333076413
0.0 -0.175 7.0 3.461778189872766 24 2.17045228498 1.12610230165 3.28773314633
0.0 -0.15 6.0 2.961926806880313 24 1.8876259284 0.993704960827 2.87714109885
0.0 -0.125 5.0 2.462132939579414 24 1.58242661786 0.845793748625 2.426515843
0.0 -0.1 4.0 1.96243798724309 24 1.26295898599 0.687833978016 1.95023457982
0.0 -0.075 3.0 1.4629356229845758 24 0.935880839767 0.524117240652 1.45986733884
0.0 -0.05 2.0 0.9638923583513113 24 0.606148917868 0.357312890443 0.963445162776
0.0 -0.025 1.0 0.46648751185583304 24 0.278698347204 0.187771343555 0.466469216705
0.0 0 0.0 0.0 24 0.0 2.24699312434e-14 2.24699312434e-14
0.0 0.025 1.0 0.46648751185583304 24 0.278698347204 0.187771343555 0.466469216705
0.0 0.05 2.0 0.9638923583513113 24 0.606148917868 0.357312890443 0.96344516

0.225 0.125 455.0 435.3347253137734 24 84.48479108 3.25929495378 87.7011426784
0.225 0.15 456.0 438.5298036359037 24 84.9198560174 2.76775260307 87.6439073428
0.225 0.175 457.0 440.98906225880734 24 85.234147588 2.40664267406 87.5943624339
0.225 0.2 458.0 442.97764942959384 24 85.4717341555 2.12593446282 87.5472427454
0.225 0.225 459.0 444.64698147899156 24 85.6579016471 1.89767046319 87.5006200636
0.225 0.25 460.0 446.0898167993867 24 85.8081269447 1.70574158898 87.454663236
0.25 -0.25 510.0 11.442464304679357 24 2.82180185911 1.79067298851 3.91110882096
0.25 -0.225 509.0 11.62684694386667 24 2.65030018489 1.99426298451 3.89003430878
0.25 -0.2 508.0 11.975293525384195 24 2.43955812782 2.23326867022 3.87308792778
0.25 -0.175 507.0 12.554833884015265 24 2.1931693569 2.52375472401 3.888692323
0.25 -0.15 506.0 13.474768789702345 24 1.9186111283 2.89414944652 3.98523775754
0.25 -0.125 505.0 14.926228162873986 24 1.62635779373 3.39565115731 4.24041453119
0.25 -0.1 504.0 17.276381838341813 2

In [13]:
""" 一次元壁体構成から周波数応答計算を経て有効熱容量を算定 """
""" 計算その10, 自立循環型住宅モデル,H11相当,付加RC厚さ変更 """ 

d_rcs = [0.0,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.10,0.11,0.12,0.13,0.14,0.15]
for d_rc in d_rcs:
    
    # 床（外皮）, 断面1
    AAA = [52.33592]     # 表面積[m2]
    HHH = [0.7]
    Wtype = [0]                                                     # 壁のタイプ:0 → 外壁扱い
    T_psi = [0.0]
    if d_rc > 0:
        NNN = [5]     # 層数N
        d_ns = [1, d_rc, 0.012, 0.08, 1]     # 厚さd[m]
        lambda_ns = [1/0.149253731343284, 1.6, 0.16, 0.045, 1/0.149253731343284]     #熱伝導率λ[W/(mK)]
        c_rho_ns =  [0, 1896.26, 715.806, 13.3952, 0]     # 容積比熱cρ[kJ/(m3K)] 
    else:
        NNN = [4]     # 層数N
        d_ns = [1, 0.012, 0.08, 1]     # 厚さd[m]
        lambda_ns = [1/0.149253731343284, 0.16, 0.045, 1/0.149253731343284]     #熱伝導率λ[W/(mK)]
        c_rho_ns =  [0, 715.806, 13.3952, 0]     # 容積比熱cρ[kJ/(m3K)] 

    # 床（外皮）, 断面2    
    AAA += [13.08398]     # 表面積[m2]
    HHH += [0.7] 
    Wtype += [0]                                                     # 壁のタイプ:0 → 外壁扱い
    T_psi += [0.0]
    if d_rc > 0:
        NNN += [5]     # 層数N
        d_ns += [1, d_rc, 0.012, 0.08, 1]     # 厚さd[m]
        lambda_ns += [1/0.149253731343284, 1.6, 0.16, 0.12, 1/0.149253731343284]     #熱伝導率λ[W/(mK)]
        c_rho_ns +=  [0, 1896.26, 715.806, 519.064, 0]     # 容積比熱cρ[kJ/(m3K)] 
    else:
        NNN += [4]     # 層数N
        d_ns += [1, 0.012, 0.08, 1]     # 厚さd[m]
        lambda_ns += [1/0.149253731343284, 0.16, 0.12, 1/0.149253731343284]     #熱伝導率λ[W/(mK)]
        c_rho_ns +=  [0, 715.806, 519.064, 0]     # 容積比熱cρ[kJ/(m3K)] 

    # 外壁, 断面1    
    AAA += [116.32573255]     # 表面積[m2]
    HHH += [1.0] 
    Wtype += [0]                                                     # 壁のタイプ:0 → 外壁扱い
    T_psi += [0.0]
    NNN += [6]     # 層数N
    d_ns += [1, 0.012, 0.075, 0.025, 0.0095, 1]     # 厚さd[m]
    lambda_ns += [1/0.10989010989011, 0.16, 0.045, 25/1000/0.09, 0.22, 1/0.10989010989011]     #熱伝導率λ[W/(mK)]
    c_rho_ns +=  [0, 715.806, 13.3952, 1.298, 904.176, 0]     # 容積比熱cρ[kJ/(m3K)] 

    # 外壁, 断面2
    AAA += [23.82575245]      #表面積[m2]
    HHH += [1.0] 
    Wtype += [0]                                                     # 壁のタイプ:0 → 外壁扱い
    T_psi += [0.0]
    NNN += [5]     # 層数N
    d_ns += [1, 0.012, 0.1, 0.0095, 1]     # 厚さd[m]
    lambda_ns += [1/0.10989010989011, 0.16, 0.12, 0.22, 1/0.10989010989011]     #熱伝導率λ[W/(mK)]
    c_rho_ns +=  [0, 715.806, 519.064, 904.176, 0]     # 容積比熱cρ[kJ/(m3K)] 

    # 間仕切壁
    AAA += [92.648]     # 表面積[m2]
    HHH += [0.0] 
    Wtype += [1]                                                     # 壁のタイプ:1 → 内壁扱い
    T_psi += [0.0]
    NNN += [5]     # 層数N
    d_ns += [1, 0.0125, 0.1, 0.0125, 1]     # 厚さd[m]
    lambda_ns += [1/0.10989010989011, 0.22, 100/1000/0.09, 0.22, 1/0.10989010989011]     #熱伝導率λ[W/(mK)]
    c_rho_ns +=  [0, 904.176, 1.298, 904.176, 0]     # 容積比熱cρ[kJ/(m3K)] 

    # 室内天井
    AAA += [52.1703]     # 表面積[m2]
    HHH += [0.0] 
    Wtype += [1]                                                     # 壁のタイプ:1 → 内壁扱い
    T_psi += [0.0]
    NNN += [3]     # 層数N
    d_ns += [1, 0.0125, 1]     # 厚さd[m]
    lambda_ns += [1/0.0900900900900901, 0.22, 1/0.0900900900900901]     #熱伝導率λ[W/(mK)]
    c_rho_ns +=  [0, 904.176, 0]     # 容積比熱cρ[kJ/(m3K)] 

    # 天井（外皮）
    AAA += [67.9042]     # 表面積[m2]
    HHH += [1.0] 
    Wtype += [0]                                                     # 壁のタイプ:0 → 外壁扱い
    T_psi += [0.0]
    NNN += [4]     # 層数N
    d_ns += [1, 0.175, 0.0095, 1]     # 厚さd[m]
    lambda_ns += [1/0.0900900900900901, 0.052, 0.22, 1/0.0900900900900901]     #熱伝導率λ[W/(mK)]
    c_rho_ns +=  [0, 10.8836, 904.176, 0]     # 容積比熱cρ[kJ/(m3K)] 

    # 室内床
    AAA += [49.27195]     # 表面積[m2]
    HHH += [0.0] 
    Wtype += [1]                                                     # 壁のタイプ:1 → 内壁扱い
    T_psi += [0.0]
    if d_rc > 0:
        NNN += [4]     # 層数N
        d_ns += [1, d_rc, 0.012, 1]     # 厚さd[m]
        lambda_ns += [1/0.149253731343284, 1.6, 0.16, 1/0.149253731343284]     #熱伝導率λ[W/(mK)]
        c_rho_ns +=  [0, 1896.26, 715.806, 0]     # 容積比熱cρ[kJ/(m3K)] 
    else:
        NNN += [3]     # 層数N
        d_ns += [1, 0.012, 1]     # 厚さd[m]
        lambda_ns += [1/0.149253731343284, 0.16, 1/0.149253731343284]     #熱伝導率λ[W/(mK)]
        c_rho_ns +=  [0, 715.806, 0]     # 容積比熱cρ[kJ/(m3K)] 

    # 基礎壁
    AAA += [0.637]     # 表面積[m2]
    HHH += [1.0] 
    Wtype += [0]                                                     # 壁のタイプ:0 → 外壁扱い
    T_psi += [0.0]
    NNN += [4]     # 層数N
    d_ns += [1, 0.12, 0.022, 1]     # 厚さd[m]
    lambda_ns += [1/0.10989010989011, 1.6, 0.028, 1/0.10989010989011]     #熱伝導率λ[W/(mK)]
    c_rho_ns +=  [0, 1896.26, 25.116, 0]     # 容積比熱cρ[kJ/(m3K)] 

    # 土間床
    AAA += [2.4843]     # 表面積[m2]
    HHH += [0.0] 
    Wtype += [0]                                                     # 壁のタイプ:0 → 外壁扱い
    T_psi += [0.0]
    NNN += [2]     # 層数N
    d_ns += [1, 0.12]     # 厚さd[m]
    lambda_ns += [1/0.149253731343284, 1.6]     #熱伝導率λ[W/(mK)]
    c_rho_ns +=  [0, 1896.26]     # 容積比熱cρ[kJ/(m3K)] 

    # 窓、ドア
    AAA += [32.2025]     # 表面積[m2]
    HHH += [1.0] 
    Wtype += [0]                                                     # 壁のタイプ:0 → 外壁扱い
    T_psi += [0.0]
    NNN += [1]     # 層数N
    d_ns += [1]     # 厚さd[m]
    lambda_ns += [3.49]     #熱伝導率λ[W/(mK)]
    c_rho_ns +=  [0]     # 容積比熱cρ[kJ/(m3K)] 

    TT = [24]     # 計算する周期T[hour]

    [Atotal,Atotal1,CC,CCsta,UUA,TT,CCAI,CCT,CCI,QAIC,QAIS,QTC,QTS] \
        = calc_specific_heat_capacity(AAA, Wtype, HHH, T_psi, NNN, d_ns, lambda_ns, c_rho_ns, TT)
        
    print(d_rc,Atotal,Atotal1,CC,CCsta,UUA,TT[0],CCAI[0],CCT[0],CCI[0])
    

0.0 502.889635 308.79938500000003 9468.75512957672 2611.699229119549 0.7565066533517304 24 5197.3673351 1081.58834029 6255.83722919
0.01 502.889635 308.79938500000003 11643.61080438672 3403.3903525870414 0.7561765964898598 24 7082.19325312 1165.8103537 8227.96342813
0.02 502.889635 308.79938500000003 13818.466479196723 4192.64968738442 0.7558494847915342 24 8712.45838275 1234.72035661 9934.58603707
0.03 502.889635 308.79938500000003 15993.322154006722 4979.504716186253 0.7555252744747476 24 10022.4862412 1283.46425658 11299.7098221
0.04 502.889635 308.79938500000003 18168.177828816722 5763.98245452005 0.7552039226895215 24 11026.6435738 1313.95525004 12338.4209193
0.05 502.889635 308.79938500000003 20343.03350362672 6546.109461859366 0.7548853874920806 24 11773.0492363 1330.75355008 13103.5230755
0.06 502.889635 308.79938500000003 22517.889178436722 7325.9118523740235 0.7545696278199038 24 12315.2969284 1338.3108424 13653.4997714
0.07 502.889635 308.79938500000003 24692.744853246724 81

In [14]:
""" 一次元壁体構成から周波数応答計算を経て有効熱容量を算定 """
""" 計算その10の2, 自立循環型住宅モデル,H04相当,付加RC厚さ変更 """ 

d_rcs = [0.0,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.10,0.11,0.12,0.13,0.14,0.15]
for d_rc in d_rcs:
    
    # 床（外皮）, 断面1
    AAA = [52.33592]     # 表面積[m2]
    HHH = [0.7]
    Wtype = [0]                                                     # 壁のタイプ:0 → 外壁扱い
    T_psi = [0.0]
    if d_rc > 0:
        NNN = [5]     # 層数N
        d_ns = [1, d_rc, 0.012, 0.047, 1]     # 厚さd[m]
        #                       ^^^^^ ←断熱材厚さ変更
        lambda_ns = [1/0.149253731343284, 1.6, 0.16, 0.045, 1/0.149253731343284]     #熱伝導率λ[W/(mK)]
        c_rho_ns =  [0, 1896.26, 715.806, 13.3952, 0]     # 容積比熱cρ[kJ/(m3K)] 
    else:
        NNN = [4]     # 層数N
        d_ns = [1, 0.012, 0.047, 1]     # 厚さd[m]
        #                 ^^^^^ ←断熱材厚さ変更
        lambda_ns = [1/0.149253731343284, 0.16, 0.045, 1/0.149253731343284]     #熱伝導率λ[W/(mK)]
        c_rho_ns =  [0, 715.806, 13.3952, 0]     # 容積比熱cρ[kJ/(m3K)] 

    # 床（外皮）, 断面2    
    AAA += [13.08398]     # 表面積[m2]
    HHH += [0.7] 
    Wtype += [0]                                                     # 壁のタイプ:0 → 外壁扱い
    T_psi += [0.0]
    if d_rc > 0:
        NNN += [5]     # 層数N
        d_ns += [1, d_rc, 0.012, 0.047, 1]     # 厚さd[m]
        #                        ^^^^^ ←断熱材厚さに相当する木材の厚を変更        
        lambda_ns += [1/0.149253731343284, 1.6, 0.16, 0.12, 1/0.149253731343284]     #熱伝導率λ[W/(mK)]
        c_rho_ns +=  [0, 1896.26, 715.806, 519.064, 0]     # 容積比熱cρ[kJ/(m3K)] 
    else:
        NNN += [4]     # 層数N
        d_ns += [1, 0.012, 0.047, 1]     # 厚さd[m]
        #                  ^^^^^ ←断熱材厚さに相当する木材の厚を変更        
        lambda_ns += [1/0.149253731343284, 0.16, 0.12, 1/0.149253731343284]     #熱伝導率λ[W/(mK)]
        c_rho_ns +=  [0, 715.806, 519.064, 0]     # 容積比熱cρ[kJ/(m3K)] 

    # 外壁, 断面1    
    AAA += [116.32573255]     # 表面積[m2]
    HHH += [1.0] 
    Wtype += [0]                                                     # 壁のタイプ:0 → 外壁扱い
    T_psi += [0.0]
    NNN += [6]     # 層数N
    d_ns += [1, 0.012, 0.048, 0.052, 0.0095, 1]     # 厚さd[m]
    #                  ^^^^^ ^^^^^ ←断熱材厚さと中空層厚さを変更        
    lambda_ns += [1/0.10989010989011, 0.16, 0.05, 52/1000/0.09, 0.22, 1/0.10989010989011]     #熱伝導率λ[W/(mK)]
    #                                       ^^^^  ^^^^^^^^^^^^ ←断熱材と中空層のλを変更    
    c_rho_ns +=  [0, 715.806, 8.372, 1.298, 904.176, 0]     # 容積比熱cρ[kJ/(m3K)] 
    #                         ^^^^^  ←断熱材の容積比熱を変更    
    
    # 外壁, 断面2
    AAA += [23.82575245]      #表面積[m2]
    HHH += [1.0] 
    Wtype += [0]                                                     # 壁のタイプ:0 → 外壁扱い
    T_psi += [0.0]
    NNN += [5]     # 層数N
    d_ns += [1, 0.012, 0.1, 0.0095, 1]     # 厚さd[m]
    lambda_ns += [1/0.10989010989011, 0.16, 0.12, 0.22, 1/0.10989010989011]     #熱伝導率λ[W/(mK)]
    c_rho_ns +=  [0, 715.806, 519.064, 904.176, 0]     # 容積比熱cρ[kJ/(m3K)] 

    # 間仕切壁
    AAA += [92.648]     # 表面積[m2]
    HHH += [0.0] 
    Wtype += [1]                                                     # 壁のタイプ:1 → 内壁扱い
    T_psi += [0.0]
    NNN += [5]     # 層数N
    d_ns += [1, 0.0125, 0.1, 0.0125, 1]     # 厚さd[m]
    lambda_ns += [1/0.10989010989011, 0.22, 100/1000/0.09, 0.22, 1/0.10989010989011]     #熱伝導率λ[W/(mK)]
    c_rho_ns +=  [0, 904.176, 1.298, 904.176, 0]     # 容積比熱cρ[kJ/(m3K)] 

    # 室内天井
    AAA += [52.1703]     # 表面積[m2]
    HHH += [0.0] 
    Wtype += [1]                                                     # 壁のタイプ:1 → 内壁扱い
    T_psi += [0.0]
    NNN += [3]     # 層数N
    d_ns += [1, 0.0125, 1]     # 厚さd[m]
    lambda_ns += [1/0.0900900900900901, 0.22, 1/0.0900900900900901]     #熱伝導率λ[W/(mK)]
    c_rho_ns +=  [0, 904.176, 0]     # 容積比熱cρ[kJ/(m3K)] 

    # 天井（外皮）
    AAA += [67.9042]     # 表面積[m2]
    HHH += [1.0] 
    Wtype += [0]                                                     # 壁のタイプ:0 → 外壁扱い
    T_psi += [0.0]
    NNN += [4]     # 層数N
    d_ns += [1, 0.062, 0.0095, 1]     # 厚さd[m]
    #           ^^^^^ ←断熱材厚さ変更        
    lambda_ns += [1/0.0900900900900901, 0.05, 0.22, 1/0.0900900900900901]     #熱伝導率λ[W/(mK)]
    #                                   ^^^^ ←断熱材のλ変更
    c_rho_ns +=  [0, 8.372, 904.176, 0]     # 容積比熱cρ[kJ/(m3K)] 
    #                ^^^^^ ←断熱材cρ変更

    # 室内床
    AAA += [49.27195]     # 表面積[m2]
    HHH += [0.0] 
    Wtype += [1]                                                     # 壁のタイプ:1 → 内壁扱い
    T_psi += [0.0]
    if d_rc > 0:
        NNN += [4]     # 層数N
        d_ns += [1, d_rc, 0.012, 1]     # 厚さd[m]
        lambda_ns += [1/0.149253731343284, 1.6, 0.16, 1/0.149253731343284]     #熱伝導率λ[W/(mK)]
        c_rho_ns +=  [0, 1896.26, 715.806, 0]     # 容積比熱cρ[kJ/(m3K)] 
    else:
        NNN += [3]     # 層数N
        d_ns += [1, 0.012, 1]     # 厚さd[m]
        lambda_ns += [1/0.149253731343284, 0.16, 1/0.149253731343284]     #熱伝導率λ[W/(mK)]
        c_rho_ns +=  [0, 715.806, 0]     # 容積比熱cρ[kJ/(m3K)] 

    # 基礎壁
    AAA += [0.637]     # 表面積[m2]
    HHH += [1.0] 
    Wtype += [0]                                                     # 壁のタイプ:0 → 外壁扱い
    T_psi += [0.0]
    NNN += [3]     # 層数N
    #       ^ ← H11から断熱材を外した構成に変更
    d_ns += [1, 0.12, 1]     # 厚さd[m]
    lambda_ns += [1/0.10989010989011, 1.6, 1/0.10989010989011]     #熱伝導率λ[W/(mK)]
    c_rho_ns +=  [0, 1896.26, 0]     # 容積比熱cρ[kJ/(m3K)] 

    # 土間床
    AAA += [2.4843]     # 表面積[m2]
    HHH += [0.0] 
    Wtype += [0]                                                     # 壁のタイプ:0 → 外壁扱い
    T_psi += [0.0]
    NNN += [2]     # 層数N
    d_ns += [1, 0.12]     # 厚さd[m]
    lambda_ns += [1/0.149253731343284, 1.6]     #熱伝導率λ[W/(mK)]
    c_rho_ns +=  [0, 1896.26]     # 容積比熱cρ[kJ/(m3K)] 

    # 窓、ドア
    AAA += [32.2025]     # 表面積[m2]
    HHH += [1.0] 
    Wtype += [0]                                                     # 壁のタイプ:0 → 外壁扱い
    T_psi += [0.0]
    NNN += [1]     # 層数N
    d_ns += [1]     # 厚さd[m]
    lambda_ns += [4.65]     #熱伝導率λ[W/(mK)]
    c_rho_ns +=  [0]     # 容積比熱cρ[kJ/(m3K)] 

    TT = [24]     # 計算する周期T[hour]

    [Atotal,Atotal1,CC,CCsta,UUA,TT,CCAI,CCT,CCI,QAIC,QAIS,QTC,QTS] \
        = calc_specific_heat_capacity(AAA, Wtype, HHH, T_psi, NNN, d_ns, lambda_ns, c_rho_ns, TT)
        
    print(d_rc,Atotal,Atotal1,CC,CCsta,UUA,TT[0],CCAI[0],CCT[0],CCI[0])
    

0.0 502.889635 308.79938500000003 9061.023251789018 2396.1844033316056 1.1018916465665476 24 4978.55517362 1112.83411589 6073.12592025
0.01 502.889635 308.79938500000003 11235.878926599016 3153.378040149708 1.1012104333225268 24 6815.45440056 1244.79277787 8044.75770958
0.02 502.889635 308.79938500000003 13410.734601409018 3907.3732761779556 1.1005374763810822 24 8420.00515195 1357.58737897 9767.61553814
0.03 502.889635 308.79938500000003 15585.590276219018 4658.220802178599 1.0998726131352152 24 9724.60327243 1442.58577513 11161.8343763
0.04 502.889635 308.79938500000003 17760.445951029014 5405.970148748886 1.0992156855232478 24 10735.3779972 1500.92803399 12234.0543131
0.05 502.889635 308.79938500000003 19935.301625839013 6150.669722352518 1.0985665398632958 24 11493.2250917 1538.11891721 13030.7901026
0.06 502.889635 308.79938500000003 22110.157300649018 6892.366839910411 1.09792502669515 24 12047.6122623 1560.1511233 13607.7605918
0.07 502.889635 308.79938500000003 24285.0129754590

In [15]:
""" 一次元壁体構成から周波数応答計算を経て有効熱容量を算定 """
""" 計算その10の3, 自立循環型住宅モデル,H11超相当,付加RC厚さ変更 """ 

d_rcs = [0.0,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.10,0.11,0.12,0.13,0.14,0.15]
for d_rc in d_rcs:
    
    # 床（外皮）, 断面1
    AAA = [52.33592]     # 表面積[m2]
    HHH = [0.7]
    Wtype = [0]                                                     # 壁のタイプ:0 → 外壁扱い
    T_psi = [0.0]
    if d_rc > 0:
        NNN = [5]     # 層数N
        d_ns = [1, d_rc, 0.012, 0.12, 1]     # 厚さd[m]
        #                       ^^^^ ← 断熱材厚さ変更
        lambda_ns = [1/0.149253731343284, 1.6, 0.16, 0.038, 1/0.149253731343284]     #熱伝導率λ[W/(mK)]
        #                                            ^^^^^ ← 断熱材のλ変更       
        c_rho_ns =  [0, 1896.26, 715.806, 20.0928, 0]     # 容積比熱cρ[kJ/(m3K)] 
        #                                 ^^^^^^^ ← 断熱材のcρ変更
    else:
        NNN = [4]     # 層数N
        d_ns = [1, 0.012, 0.12, 1]     # 厚さd[m]
        #                 ^^^^ ← 断熱材厚さ変更
        lambda_ns = [1/0.149253731343284, 0.16, 0.038, 1/0.149253731343284]     #熱伝導率λ[W/(mK)]
        #                                       ^^^^^ ← 断熱材のλ変更  
        c_rho_ns =  [0, 715.806, 20.0928, 0]     # 容積比熱cρ[kJ/(m3K)] 
        #                        ^^^^^^^ ← 断熱材のcρ変更
        
    # 床（外皮）, 断面2    
    AAA += [13.08398]     # 表面積[m2]
    HHH += [0.7] 
    Wtype += [0]                                                     # 壁のタイプ:0 → 外壁扱い
    T_psi += [0.0]
    if d_rc > 0:
        NNN += [5]     # 層数N
        d_ns += [1, d_rc, 0.012, 0.12, 1]     # 厚さd[m]
        #                        ^^^^ ← 断熱材厚さに対応する木材の厚さ変更
        lambda_ns += [1/0.149253731343284, 1.6, 0.16, 0.12, 1/0.149253731343284]     #熱伝導率λ[W/(mK)]
        c_rho_ns +=  [0, 1896.26, 715.806, 519.064, 0]     # 容積比熱cρ[kJ/(m3K)] 
    else:
        NNN += [4]     # 層数N
        d_ns += [1, 0.012, 0.12, 1]     # 厚さd[m]
        #                  ^^^^ ← 断熱材厚さに対応する木材の厚さ変更
        lambda_ns += [1/0.149253731343284, 0.16, 0.12, 1/0.149253731343284]     #熱伝導率λ[W/(mK)]
        c_rho_ns +=  [0, 715.806, 519.064, 0]     # 容積比熱cρ[kJ/(m3K)] 

    # 外壁, 断面1    
    AAA += [116.32573255]     # 表面積[m2]
    HHH += [1.0] 
    Wtype += [0]                                                     # 壁のタイプ:0 → 外壁扱い
    T_psi += [0.0]
    NNN += [5]     # 層数N
    #       ^ ← 中空層なし→層数変更
    d_ns += [1, 0.012, 0.121, 0.0095, 1]     # 厚さd[m]
    #                  ^^^^^ ← 断熱材厚さ変更、中空層なし
    lambda_ns += [1/0.10989010989011, 0.16, 0.038, 0.22, 1/0.10989010989011]     #熱伝導率λ[W/(mK)]
    #                                       ^^^^^ ← 断熱材λ変更、中空層なし
    c_rho_ns +=  [0, 715.806, 20.0928, 904.176, 0]     # 容積比熱cρ[kJ/(m3K)] 
    #                         ^^^^^^^ ← 断熱材cρ変更、中空層なし
    
    # 外壁, 断面2
    AAA += [23.82575245]      #表面積[m2]
    HHH += [1.0] 
    Wtype += [0]                                                     # 壁のタイプ:0 → 外壁扱い
    T_psi += [0.0]
    NNN += [5]     # 層数N
    d_ns += [1, 0.012, 0.121, 0.0095, 1]     # 厚さd[m]
    #                  ^^^^^ ← 断熱材にあわせて厚さ変更
    lambda_ns += [1/0.10989010989011, 0.16, 0.12, 0.22, 1/0.10989010989011]     #熱伝導率λ[W/(mK)]
    c_rho_ns +=  [0, 715.806, 519.064, 904.176, 0]     # 容積比熱cρ[kJ/(m3K)] 

    # 間仕切壁
    AAA += [92.648]     # 表面積[m2]
    HHH += [0.0] 
    Wtype += [1]                                                     # 壁のタイプ:1 → 内壁扱い
    T_psi += [0.0]
    NNN += [5]     # 層数N
    d_ns += [1, 0.0125, 0.1, 0.0125, 1]     # 厚さd[m]
    lambda_ns += [1/0.10989010989011, 0.22, 100/1000/0.09, 0.22, 1/0.10989010989011]     #熱伝導率λ[W/(mK)]
    c_rho_ns +=  [0, 904.176, 1.298, 904.176, 0]     # 容積比熱cρ[kJ/(m3K)] 

    # 室内天井
    AAA += [52.1703]     # 表面積[m2]
    HHH += [0.0] 
    Wtype += [1]                                                     # 壁のタイプ:1 → 内壁扱い
    T_psi += [0.0]
    NNN += [3]     # 層数N
    d_ns += [1, 0.0125, 1]     # 厚さd[m]
    lambda_ns += [1/0.0900900900900901, 0.22, 1/0.0900900900900901]     #熱伝導率λ[W/(mK)]
    c_rho_ns +=  [0, 904.176, 0]     # 容積比熱cρ[kJ/(m3K)] 

    # 天井（外皮）
    AAA += [67.9042]     # 表面積[m2]
    HHH += [1.0] 
    Wtype += [0]                                                     # 壁のタイプ:0 → 外壁扱い
    T_psi += [0.0]
    NNN += [4]     # 層数N
    d_ns += [1, 0.285, 0.0095, 1]     # 厚さd[m]
    #           ^^^^^ ← 断熱材厚さ変更
    lambda_ns += [1/0.0900900900900901, 0.052, 0.22, 1/0.0900900900900901]     #熱伝導率λ[W/(mK)]
    c_rho_ns +=  [0, 10.8836, 904.176, 0]     # 容積比熱cρ[kJ/(m3K)] 

    # 室内床
    AAA += [49.27195]     # 表面積[m2]
    HHH += [0.0] 
    Wtype += [1]                                                     # 壁のタイプ:1 → 内壁扱い
    T_psi += [0.0]
    if d_rc > 0:
        NNN += [4]     # 層数N
        d_ns += [1, d_rc, 0.012, 1]     # 厚さd[m]
        lambda_ns += [1/0.149253731343284, 1.6, 0.16, 1/0.149253731343284]     #熱伝導率λ[W/(mK)]
        c_rho_ns +=  [0, 1896.26, 715.806, 0]     # 容積比熱cρ[kJ/(m3K)] 
    else:
        NNN += [3]     # 層数N
        d_ns += [1, 0.012, 1]     # 厚さd[m]
        lambda_ns += [1/0.149253731343284, 0.16, 1/0.149253731343284]     #熱伝導率λ[W/(mK)]
        c_rho_ns +=  [0, 715.806, 0]     # 容積比熱cρ[kJ/(m3K)] 

    # 基礎壁
    AAA += [0.637]     # 表面積[m2]
    HHH += [1.0] 
    Wtype += [0]                                                     # 壁のタイプ:0 → 外壁扱い
    T_psi += [0.0]
    NNN += [4]     # 層数N
    d_ns += [1, 0.12, 0.034, 1]     # 厚さd[m]
    #                 ^^^^^ ← 断熱材厚さ変更
    lambda_ns += [1/0.10989010989011, 1.6, 0.028, 1/0.10989010989011]     #熱伝導率λ[W/(mK)]
    c_rho_ns +=  [0, 1896.26, 25.116, 0]     # 容積比熱cρ[kJ/(m3K)] 

    # 土間床
    AAA += [2.4843]     # 表面積[m2]
    HHH += [0.0] 
    Wtype += [0]                                                     # 壁のタイプ:0 → 外壁扱い
    T_psi += [0.0]
    NNN += [2]     # 層数N
    d_ns += [1, 0.12]     # 厚さd[m]
    lambda_ns += [1/0.149253731343284, 1.6]     #熱伝導率λ[W/(mK)]
    c_rho_ns +=  [0, 1896.26]     # 容積比熱cρ[kJ/(m3K)] 

    # 窓、ドア
    AAA += [32.2025]     # 表面積[m2]
    HHH += [1.0] 
    Wtype += [0]                                                     # 壁のタイプ:0 → 外壁扱い
    T_psi += [0.0]
    NNN += [1]     # 層数N
    d_ns += [1]     # 厚さd[m]
    lambda_ns += [3.49]     #熱伝導率λ[W/(mK)]
    c_rho_ns +=  [0]     # 容積比熱cρ[kJ/(m3K)] 

    TT = [24]     # 計算する周期T[hour]

    [Atotal,Atotal1,CC,CCsta,UUA,TT,CCAI,CCT,CCI,QAIC,QAIS,QTC,QTS] \
        = calc_specific_heat_capacity(AAA, Wtype, HHH, T_psi, NNN, d_ns, lambda_ns, c_rho_ns, TT)
        
    print(d_rc,Atotal,Atotal1,CC,CCsta,UUA,TT[0],CCAI[0],CCT[0],CCI[0])
    

0.0 502.889635 308.79938500000003 10313.886799201342 2995.0702561972753 0.6233156163069439 24 5432.26766176 1085.55274656 6477.21666192
0.01 502.889635 308.79938500000003 12488.742474011342 3812.2064994540183 0.6231584482742701 24 7345.71921333 1131.68549701 8441.02674179
0.02 502.889635 308.79938500000003 14663.598148821342 4627.628866004728 0.6230023717587135 24 8991.16201681 1167.59073296 10134.5200856
0.03 502.889635 308.79938500000003 16838.453823631342 5441.351289548226 0.6228473737741363 24 10303.4074802 1190.86321402 11481.092907
0.04 502.889635 308.79938500000003 19013.30949844134 6253.387518718505 0.6226934415545511 24 11302.5738909 1203.22790079 12500.2205035
0.05 502.889635 308.79938500000003 21188.16517325134 7063.751120642616 0.6225405625493274 24 12041.5528071 1207.84312266 13247.9763744
0.06 502.889635 308.79938500000003 23363.02084806134 7872.4554844098575 0.6223887244185263 24 12576.3949113 1207.57360722 13783.9494936
0.07 502.889635 308.79938500000003 25537.876522871

In [21]:
""" 一次元壁体構成から周波数応答計算を経て有効熱容量を算定 """
""" 計算その11の1, 「RC150mm床」単体 """ 

AAA = [1.05]                                                       # 複数躯体構成｢1組｣のそれぞれの表面積[m2]
HHH = [0.0]                                                     # 複数躯体構成を介して隣接する空間に適用する温度差係数[-]→外気：Hm = 1
T_psi = [0.0]                                                   # 複数躯体構成を介して隣接する空間との温度変動の位相のずれ[hour]
                                                                  # →原則ずれはなし：T_psi = 0
NNN = [3]                                                       # 複数躯体構成｢1組｣のそれぞれの層数N
d_ns =      [1,      0.15,    1]                      # 1組目の各層の厚さd[m]
lambda_ns = [1/0.09, 1.60,    1/0.09]                 # 1組目の各層の熱伝導率λ[W/(mK)]
c_rho_ns =  [0,      1896.260,0]                      # 1組目の各層の容積比熱cρ[kJ/(m3K)] 
TT = [24]                                                       # 計算する周期T[hour]
Wtype = [1]                                                     # 壁のタイプ:1 → 内壁扱い

[Atotal,Atotal1,CC,CCsta,UUA,TT,CCAI,CCT,CCI,QAIC,QAIS,QTC,QTS] \
    = calc_specific_heat_capacity(AAA, Wtype, HHH, T_psi, NNN, d_ns, lambda_ns, c_rho_ns, TT)

print (Atotal,Atotal1,CC,CCsta,UUA,TT,CCAI,CCT,CCI,QAIC,QAIS,QTC,QTS)
print ('T = {} [hour] におけるa側有効熱容量 Ci = {} [kJ/(m2K)]'.format( TT[0], CCI[0] ))
print ('T = {} [hour] におけるa側有効吸熱熱容量 CAi = {} [kJ/(m2K)]'.format( TT[0], CCAI[0] ))
print ('T = {} [hour] における有効貫流熱容量 CT = {} [kJ/(m2K)]'.format( TT[0], CCT[0] ))
print ('C = {} [kJ/(m2K)],  Cstastic = {} [kJ/(m2K)]'.format( CC, CCsta ))

1.05 0.0 298.66094999999996 0.0 0.0 [24] [201.20547178119131] [0.0] [201.20547178119131] [10.766228630924376] [9.908881594795691] [0] [0]
T = 24 [hour] におけるa側有効熱容量 Ci = 201.2054717811913 [kJ/(m2K)]
T = 24 [hour] におけるa側有効吸熱熱容量 CAi = 201.2054717811913 [kJ/(m2K)]
T = 24 [hour] における有効貫流熱容量 CT = 0.0 [kJ/(m2K)]
C = 298.66094999999996 [kJ/(m2K)],  Cstastic = 0.0 [kJ/(m2K)]


In [20]:
""" 一次元壁体構成から周波数応答計算を経て有効熱容量を算定 """
""" 計算その11の2, 「RC150mm+内断熱50mm」外壁""" 

AAA = [2.25]                                                       # 複数躯体構成｢1組｣のそれぞれの表面積[m2]
HHH = [1.0]                                                     # 複数躯体構成を介して隣接する空間に適用する温度差係数[-]→外気：Hm = 1
T_psi = [0.0]                                                   # 複数躯体構成を介して隣接する空間との温度変動の位相のずれ[hour]
                                                                  # →原則ずれはなし：T_psi = 0
NNN = [4]                                                       # 複数躯体構成｢1組｣のそれぞれの層数N
d_ns =      [1,      0.05,  0.15,    1]                      # 1組目の各層の厚さd[m]
lambda_ns = [1/0.09, 0.028, 1.60,    1/0.04]                 # 1組目の各層の熱伝導率λ[W/(mK)]
c_rho_ns =  [0,      25.116,1896.260,0]                      # 1組目の各層の容積比熱cρ[kJ/(m3K)] 
TT = [24]                                                       # 計算する周期T[hour]
Wtype = [0]                                                     # 壁のタイプ:0 → 外壁扱い

[Atotal,Atotal1,CC,CCsta,UUA,TT,CCAI,CCT,CCI,QAIC,QAIS,QTC,QTS] \
    = calc_specific_heat_capacity(AAA, Wtype, HHH, T_psi, NNN, d_ns, lambda_ns, c_rho_ns, TT)

print (Atotal,Atotal1,CC,CCsta,UUA,TT,CCAI,CCT,CCI,QAIC,QAIS,QTC,QTS)
print ('T = {} [hour] におけるa側有効熱容量 Ci = {} [kJ/(m2K)]'.format( TT[0], CCI[0] ))
print ('T = {} [hour] におけるa側有効吸熱熱容量 CAi = {} [kJ/(m2K)]'.format( TT[0], CCAI[0] ))
print ('T = {} [hour] における有効貫流熱容量 CT = {} [kJ/(m2K)]'.format( TT[0], CCT[0] ))
print ('C = {} [kJ/(m2K)],  Cstastic = {} [kJ/(m2K)]'.format( CC, CCsta ))

2.25 2.25 642.8133 29.112070321247703 0.497645072425131 [24] [1.4660058715713962] [15.159746505598688] [16.433339383329741] [0.047062565491440084] [0.095660915505313998] [0.92695885664147948] [0.5967734895702721]
T = 24 [hour] におけるa側有効熱容量 Ci = 16.43333938332974 [kJ/(m2K)]
T = 24 [hour] におけるa側有効吸熱熱容量 CAi = 1.4660058715713962 [kJ/(m2K)]
T = 24 [hour] における有効貫流熱容量 CT = 15.159746505598688 [kJ/(m2K)]
C = 642.8133 [kJ/(m2K)],  Cstastic = 29.112070321247703 [kJ/(m2K)]


In [18]:
""" 一次元壁体構成から周波数応答計算を経て有効熱容量を算定 """
""" 計算その11の3, 「RC150mm+内断熱50mm」外壁2.25m2 +「RC150mm床」1.05m2 """ 

# 外壁2.25m2
AAA = [2.25]                                                    # 複数躯体構成｢1組｣のそれぞれの表面積[m2]
HHH = [1.0]                                                     # 複数躯体構成を介して隣接する空間に適用する温度差係数[-]→外気：Hm = 1
Wtype = [0]                                                     # 壁のタイプ:0 → 外壁扱い
T_psi = [0.0]                                                   # 複数躯体構成を介して隣接する空間との温度変動の位相のずれ[hour]
                                                                  # →原則ずれはなし：T_psi = 0
NNN = [4]                                                       # 複数躯体構成｢1組｣のそれぞれの層数N
d_ns =      [1,      0.05,  0.15,    1]                      # 1組目の各層の厚さd[m]
lambda_ns = [1/0.09, 0.028, 1.60,    1/0.04]                 # 1組目の各層の熱伝導率λ[W/(mK)]
c_rho_ns =  [0,      25.116,1896.260,0]                      # 1組目の各層の容積比熱cρ[kJ/(m3K)] 

# 床1.05m2
AAA += [1.05]                                                    # 複数躯体構成｢1組｣のそれぞれの表面積[m2]
HHH += [0.0]                                                     # 複数躯体構成を介して隣接する空間に適用する温度差係数[-]→外気：Hm = 1
Wtype += [1]                                                     # 壁のタイプ:1 → 内壁扱い
T_psi += [0.0]                                                   # 複数躯体構成を介して隣接する空間との温度変動の位相のずれ[hour]
                                                                  # →原則ずれはなし：T_psi = 0
NNN += [3]                                                       # 複数躯体構成｢1組｣のそれぞれの層数N
d_ns +=      [1,      0.15,    1]                      # 1組目の各層の厚さd[m]
lambda_ns += [1/0.09, 1.60,    1/0.09]                 # 1組目の各層の熱伝導率λ[W/(mK)]
c_rho_ns +=  [0,      1896.260,0]                      # 1組目の各層の容積比熱cρ[kJ/(m3K)] 

TT = [24]                                                       # 計算する周期T[hour]

[Atotal,Atotal1,CC,CCsta,UUA,TT,CCAI,CCT,CCI,QAIC,QAIS,QTC,QTS] \
        = calc_specific_heat_capacity(AAA, Wtype, HHH, T_psi, NNN, d_ns, lambda_ns, c_rho_ns, TT)

print (Atotal,Atotal1,CC,CCsta,UUA,TT,CCAI,CCT,CCI,QAIC,QAIS,QTC,QTS)
print ('T = {} [hour] におけるa側有効熱容量 Ci = {} [kJ/(mK)]'.format( TT[0], CCI[0] ))
print ('T = {} [hour] におけるa側有効吸熱熱容量 CAi = {} [kJ/(mK)]'.format( TT[0], CCAI[0] ))
print ('T = {} [hour] における有効貫流熱容量 CT = {} [kJ/(mK)]'.format( TT[0], CCT[0] ))
print ('C = {} [kJ/(m2K)],  Cstastic = {} [kJ/(mK)]'.format( CC, CCsta ))


3.3 2.25 941.47425 29.112070321247703 0.497645072425131 [24] [202.57315397913695] [15.159746505598688] [217.51843746709753] [10.813291196415815] [10.004542510301004] [0.92695885664147948] [0.5967734895702721]
T = 24 [hour] におけるa側有効熱容量 Ci = 217.51843746709753 [kJ/(mK)]
T = 24 [hour] におけるa側有効吸熱熱容量 CAi = 202.57315397913695 [kJ/(mK)]
T = 24 [hour] における有効貫流熱容量 CT = 15.159746505598688 [kJ/(mK)]
C = 941.47425 [kJ/(m2K)],  Cstastic = 29.112070321247703 [kJ/(mK)]


### 1.5 計算式の確認

- ISO 13786, Thermal performance of building components - Dynamic thermal　characteristics - Calculation methods, 2007
- 以下、「小文字アルファベット以外の」式番号は上記文献に対応。ただしダッシュ(')付きは本稿にあわせて変更している。
- 他に、永田明寛：周波数応答の選点適合に基づく外壁熱性能値, 日本建築学会学術講演梗概集D2, pp.287, 2010.9

#### (1) 一次元躯体の構成と熱物性

##### a) 対象とする空間単位

有効熱容量を求める空間は以下の単位とする。

- 住戸全体：計算対象住戸の居室、非居室と断熱区画内の空間
- 複数の空間：計算対象とする住戸の複数の居室、非居室と断熱区画内の空間
- 単一の空間：計算対象とする住戸の単一の居室、非居室と断熱区画内の空間

上記の空間(群)が面する、外界に面する躯体、隣接住戸の境界となる躯体、同一住戸内の躯体の総数を$M$($m=1$～$M$)とする。

##### b) 躯体の設定

躯体$M$について、以下の値を設定する。なお、外界に面していない躯体については、2つの表面のそれぞれの方向から別個に入力することはないものとする(例：同一住戸の1、2階間の床では、2階を$n=0$側、1階を$n=N$側(隣接する空間側)とし、壁の種別(`Wtype`)を設定することで、隣接側の励振を考慮するようにする)。

- 表面積$A_m \; [m^2]$
- 壁の種別`Wtype`：室内側有効吸熱熱容量、有効貫流熱容量の加算対象かどうかを判別するためのタイプ分け
  - `Wtype` $=0$：外界に面した躯体、隣接住戸の境界となる躯体(=隣接空間$n=N$側は対象空間ではない) → 有効貫流熱容量の加算対象。$U_A$値計算対象(面積、熱貫流率積算対象)。
  - `Wtype` $=1$：両面が対象とする空間に面した躯体 → $n=0$、$n=N$の両側が室内側有効吸熱熱容量の加算対象。有効貫流熱容量には加算されない。
  - `Wtype` $=2$：隣接空間側が対象空間外だが同一住戸等 → $n=0$側が室内側有効吸熱熱容量の加算対象。$n=N$側の室内側有効吸熱熱容量は隣接空間側で加算されるべきとして加算対象外。有効貫流熱容量には加算対象外。
  

- 躯体$m$を介して隣接する空間に適用する温度差係数$H_m \; [-]$ 
- 躯体$m$を介して隣接する空間との温度変動の位相のずれ$T_{\psi,m} \; [hour]$
  - $T_{\psi,m} <0$：隣接空間の変動が対象空間より遅れる。

温度差係数$H_m$は、「平成28年省エネルギー基準における住宅のエネルギー消費性能の算定方法第二章第二節 外皮性能」のもと、隣接空間の種類に応じて、以下の値を設定する。ただし、「*」については暫定の設定。

- 外気および外気に通じる空間：$H_m=1.0$
- 外気に通じていない空間および外気に通じる床裏：$H_m=0.7$
- 住戸及び住戸と同様の熱的環境の空間、ならびに外気に通じていない床裏：

  - $1$～$3$地域：$H_m=0.05$
  - $4$～$8$地域：$H_m=0.15$


- 同一住戸内の同様の熱的環境の空間*：$H_m=0.0$
  - `Wtype` $=1, 2$で設定

##### c) 躯体の層構成

本稿で扱う一次元躯体$m$は、$N_m-2\;$層からなる躯体構成要素とその両面の表面熱伝達層を持つ、全体で$N_m$層で構成される躯体とする。層構成を以下のように割り当てる。

- 層$1 \; (n=1)$：有効熱容量を計算する側(例：外壁の室内側)の表面熱伝達層
- 層$N_m \; (n=N_m)$：反対側(例：外壁の外気側)の表面熱伝達層
- 層$2$～層$N_m-1 \; (n=2$～$N_m-1)$：躯体を構成する部材の層(中空層含む)($n=1$側から層順に設定)

一次元躯体$m$の層$n$には、以下の物性値等を設定する。

- 厚さ$d_{m,n}\;[m]$
- 熱伝導率$\lambda_{m,n} \; [W/(m･K)]$
- 容積比熱$(c\rho)_{m,n} \; [kJ/(m^3･K)]$

  - 表面熱伝達層や中空層の場合は、$(c\rho)_{m,n} = 0\;[kJ/(m^3･K)]$とする。

  - 中空層の場合は、厚さ$d_{m,n}\;[m]$を、その中空層の熱抵抗$r_{m,n} \; [m^2K/W]$で除した値を、$\lambda_{m,n} \; [W/(m･K)]$として設定する($\lambda_{m,n} \leftarrow d_{m,n} \: / \: r_{m,n} $)。

  - 表面熱伝達層の場合は、$d_{m,n}=1\;[m]$とし、表面熱伝達率$\alpha_s \; [W/(m^2･K)]$を$\lambda_{m,n}$に設定する($\lambda_{m,n} \leftarrow \alpha_s $)。


#### (2) 熱容量と$U_A$値

##### a) 熱容量と熱貫流率

対象空間が面する躯体の熱容量$C \; [kJ/K]$は下式で求める。当初の計算式から変更し、躯体の熱容量は、入力した躯体すべてについて熱容量として加算することに変更する(温度差係数は非考慮とし、隣接空間との温度差は考慮しない → 単純に熱容量としての評価とする)。

$$ C = \sum_{m=1}^{M} \biggl( A_m \sum_{n=1}^{N_m} (c\rho)_{m,n} d_{m,n} \biggr) \ \qquad (a)$$

別に温度差$H_m$(外気とは$1K$差)の定常状態における蓄熱量を計算する。
躯体$1$～$M$の熱貫流率$U_m$は下式で求まる。

$$ U_m = 1 \bigg/ \sum_{n=1}^{N_m} \frac{ d_{m,n} }{ \lambda_{m,n} }  \qquad (b)$$

$a$側の温度$1 \: [K]$、$b$側の温度$1-H_m \: [K]$の温度差$H_m$における定常状態を考えると、層$n-1$と層$n$の境界の温度$\theta_n$の温度は、

$$\theta_n = \theta_{n-1} - U_m H_m \frac{ d_{n-1} }{ \lambda_{n-1} } = H_m - U_m H_m \sum_{n'=1}^{n-1} \frac{ d_{n'} }{ \lambda_{n'} } \qquad (c)$$

で求まる。よって、躯体$m$の温度差$H_m$の定常状態における蓄熱量$C_{static,m} \; [kJ/K]$は、躯体内の温度分布から、

$$ \begin{align}
C_{static,m} &= A_m \sum_{n=1}^{N_m} \Bigl\{ (c\rho)_{m,n} d_{m,n} \frac{ \theta_n + \theta_{n+1} }{ 2 } \Bigr\} \\
&= A_m \sum_{n=1}^{N_m} \biggl[ (c\rho)_{m,n} d_{m,n} \biggl\{ H_m - U_m H_m \biggl( \sum_{n'=1}^{n-1} \frac{ d_{n'} }{ \lambda_{n'} } + \frac{ d_n }{ 2 \lambda_n } \biggr) \biggr\} \biggr] \qquad (d) \\
\end{align} $$

と求まる。対象空間が面する躯体全体の定常状態における蓄熱量$C_{static} \; [kJ/K]$は、

$$
C_{static} = \sum_{m=1}^{M} C_{static,m} = \sum_{m=1}^{M} \biggl[ A_m \sum_{n=1}^{N_m} \biggl[ (c\rho)_{m,n} d_{m,n} \biggl\{ H_m - U_m H_m \biggl( \sum_{n'=1}^{n-1} \frac{ d_{n'} }{ \lambda_{n'} } + \frac{ d_n }{ 2 \lambda_n } \biggr) \biggr\} \biggr] \biggr] \qquad (e)\\
$$

となる。

##### b) $U_A$値

$U_A$値(ただし、熱橋等の線熱貫流率非考慮)の計算は、`Wtype`$=0$の躯体について、熱貫流率を積和して求める。

$$U_A = \sum_{m=1}^{M} \bigl( \delta_{wtype=0} \: A_m U_m H_m \bigr) \big/ \sum_{m=1}^{M} \bigl( \delta_{wtype=0} \:A_m \bigr) \qquad (e1)\\
$$ 

ただし、$\delta_{wtype=0}\:$は、`Wtype`$=0$で$1$を、それ以外の`Wtype`で$0$をとる。


#### (3) 周期

有効熱容量を計算する周期は、$T=24\:[hour]$とする。

角周波数$\omega \: [rad/s] \:$は、周期$T\: [hour]\: $から下式で求める。

$$\omega = 2 \pi \: \big/ \: (T \times 3600) \qquad (f)$$

#### (4) 単層の四端子行列の計算

- 本稿内では、下記について、入力時と計算時で単位が異なる。中途で変換されるので注意すること。
   - 時間$T$：入力時は$[hour]$立て、計算時は基本的に角周波数$[rad/s]$を用いているので表出しないが$[s]$立てとなる。
   - 容積比熱$c\rho$：入力時は$[kJ/(m^3･K)]$、計算時は$[J/(m^3･K)]$。

##### a) 躯体単層の四端子行列

$(c\rho)_{m,n} \neq 0$の単層については、角周波数$\omega\:[rad/s]$、各層の厚さ$d_{m,n}\:[m]$、熱伝導率$\lambda_{m,n}\:[W/(m･K)]$、容積比熱$(c\rho)_{m,n}\:[J/(m^3･K)]$から層$n$の単層の四端子行列$\textbf{Z}_{m,n}$を算出する。$j$は虚数単位である($\:j=\sqrt{(-1)}\:$)。

- 各層の侵入厚さ$\:\delta[m]\:$：

$$ \delta_{m,n} = \sqrt {\frac{\lambda_{m,n} T}{\pi (c\rho)_{m,n}}} = \sqrt {\frac{2 \lambda_{m,n}}{\omega (c\rho)_{m,n}}} \qquad (11') $$

- 各層の侵入厚さの比$\:\xi\:$：

$$ \xi_{m,n} = \frac{d_{m,n}}{\delta_{m,n}} = d_{m,n} \sqrt {\frac{\pi (c\rho)_{m,n}}{\lambda_{m,n} T}} = d_{m,n} \sqrt {\frac{\omega (c\rho)_{m,n}}{2 \lambda_{m,n}}} \qquad (13') $$

- 躯体単層($\:(c\rho)_{m,n} > 0\:$)の四端子行列$\;\textbf{Z}_{m,n} = \begin{pmatrix} Z_{11,m,n} & Z_{12,m,n} \\ Z_{21,m,n} & Z_{22,m,n} \end{pmatrix}\;$ の要素：

$$ \begin{align}
Z_{11,m,n} = Z_{22,m,n} &= \cosh(\xi_{m,n}) \cos(\xi_{m,n}) + j \sinh(\xi_{m,n}) \sin(\xi_{m,n}) \\ 
Z_{12,m,n} &= - \frac{\delta_{m,n}}{2 \lambda_{m,n}} \{ \sinh(\xi_{m,n}) \cos(\xi_{m,n}) + \cosh(\xi_{m,n}) \sin(\xi_{m,n}) \\
& \hspace{30pt} + j \;[ \cosh(\xi_{m,n}) \sin(\xi_{m,n}) - \sinh(\xi_{m,n}) \cos(\xi_{m,n}) ] \} \\
Z_{21,m,n} &= - \frac{\lambda_{m,n}}{\delta_{m,n}} \{\sinh(\xi_{m,n}) \cos(\xi_{m,n}) - \cosh(\xi_{m,n}) \sin(\xi_{m,n})\\ 
& \hspace{30pt} + j \; [\sinh(\xi_{m,n}) \cos(\xi_{m,n}) + \cosh(\xi_{m,n}) \sin(\xi_{m,n})]\} \qquad (14')  \\
\end{align} $$


##### b) 表面熱伝達層、中空層の四端子行列

$(c\rho)_{m,n} = 0$の表面熱伝達層、中空層については、熱抵抗$R_{m,n} = d_{m,n}\:/\:\lambda_{m,n}$を用いて、下式で単層の四端子行列$\textbf{Z}_{m,n}$を算出する。

- 表面熱伝達層、中空層($\:(c\rho)_{m,n} = 0\:$)の四端子行列：
  
$$ \;\textbf{Z}_{m,n} = \begin{pmatrix} 1 & -R_{m,n} \\ 0 & 1 \end{pmatrix} = \begin{pmatrix} 1 & -d_{m,n}/\lambda_{m,n} \\ 0 & 1 \end{pmatrix}  \qquad (15'), \; (18') $$

#### (5) 一次元躯体の四端子行列

層$1$～$N_m$の四端子行列を、各層の行列をかけあわせて算出する。

- 層$1$～$N_m$の四端子行列$\;\textbf{Z}_m\;$：

$$\textbf{Z}_m  = \begin{pmatrix} Z_{11,m} & Z_{12,m} \\ Z_{21,m} & Z_{22,m} \end{pmatrix} = \begin{pmatrix} R_{11,m}+jI_{11,m} & R_{12,m}+jI_{12,m} \\ R_{21,m}+jI_{21,m} & R_{22,m}+jI_{22,m} \end{pmatrix}=\textbf{Z}_{m,N} \; \textbf{Z}_{m,N-1} \;  \cdots \;  \textbf{Z}_{m,2} \;  \textbf{Z}_{m,1}  \qquad (16')$$

ここで、$R_{**}$は$Z_{**}$の実数部、$jI_{**}$は虚数部($I_{**}$自体は実数)。


#### (6) 熱流の周波数応答

##### a) 温度と熱流の関係式

$a$側($n=1$側)の温度変動$\hat{\theta}_{m,a}$、熱流変動$\hat{q}_{m,a}$、$b$側($n=N_m$側)の温度変動$\hat{\theta}_{m,b}$、熱流変動$\hat{q}_{m,b}$とし、$a \rightarrow b$の熱流の向きを正とすると、式$(16')$の四端子行列は、

$$\begin{pmatrix} \hat{\theta}_{m,b} \\ - \hat{q}_{m,b} \end{pmatrix}=\begin{pmatrix} Z_{11,m} & Z_{12,m} \\ Z_{21,m} & Z_{22,m} \end{pmatrix} \begin{pmatrix} \hat{\theta}_{m,a} \\ \hat{q}_{m,a} \end{pmatrix}  \qquad (12') \\ $$

の関係をみたす。これを書き直すと、以下の式を得る。

$$\begin{pmatrix} \hat{q}_{m,a} \\ - \hat{q}_{m,b} \end{pmatrix} = \frac{1}{Z_{12,m}} \begin{pmatrix} - Z_{11,m} & 1 \\ 1 & - Z_{22,m} \end{pmatrix} \begin{pmatrix} \hat{\theta}_{m,a} \\ \hat{\theta}_{m,b} \end{pmatrix}  \qquad (B.8') \\ $$

$\hat{\theta}_{m,a} = |{\theta}_{m,a}|e^{ j (\omega t + \psi_{m,a} ) }$、$\hat{\theta}_{m,b} = |{\theta}_{m,b}|e^{ j (\omega t + \psi_{m,b} ) }$、$\hat{q}_{m,a} = |{q}_{m,a}|e^{ j (\omega t + \varphi_{m,a} ) }$、$\hat{q}_{m,b} = |{q}_{m,b}|e^{ j (\omega t + \varphi_{m,b} ) }$ とすると、式$(12')$から、

$$\begin{pmatrix} |{\theta}_{m,b}|e^{ j (\omega t + \psi_{m,b} ) } \\ - |{q}_{m,b}|e^{ j (\omega t + \varphi_{m,b} ) } \end{pmatrix}=\begin{pmatrix} R_{11,m}+jI_{11,m} & R_{12,m}+jI_{12,m} \\ R_{21,m}+jI_{21,m} & R_{22,m}+jI_{22,m} \end{pmatrix} \begin{pmatrix} |{\theta}_{m,a}|e^{ j (\omega t + \psi_{m,a} ) } \\ |{q}_{m,a}|e^{ j (\omega t + \varphi_{m,a} ) } \end{pmatrix} \qquad (g) \\ $$

が、式$(B.8')$から、

$$\begin{pmatrix} |{q}_{m,a}|e^{ j (\omega t + \varphi_{m,a} ) } \\ - |{q}_{m,b}|e^{ j (\omega t + \varphi_{m,b} ) } \end{pmatrix} = \frac{1}{R_{12,m}+jI_{12,m}} \begin{pmatrix} - (R_{11,m}+jI_{11,m}) & 1 \\ 1 & - (R_{22,m}+jI_{22,m}) \end{pmatrix} \begin{pmatrix} |{\theta}_{m,a}|e^{ j (\omega t + \psi_{m,a} ) } \\ |{\theta}_{m,b}|e^{ j (\omega t + \psi_{m,b} ) } \end{pmatrix}  \qquad (h) \\ $$

が全く同じ式として得られる。

##### b) a側が単位温度振幅で励振する際の熱流応答

$\hat{\theta}_{m,a} = \cos \omega t$、$\hat{\theta}_{m,b} = 0\:$として、$a$側での単位振幅の温度励振の場合を考える。式$(g)$、$(h)$の実数部のみ考慮し、$a$側励振に対する$a$側応答$\hat{q}_{m,a,a} = |{q}_{m,a,a}|e^{ j (\omega t + \varphi_{m,a,a} ) }$、$b$側応答$\hat{q}_{m,a,b} = |{q}_{m,a,b}|e^{ j (\omega t + \varphi_{m,a,b} ) }$を求める。

$|{\theta}_{m,a}|=1$、$\psi_{m,a}=0$、$|{\theta}_{m,b}|=0$を、式$(g)$第1式に代入して、

$$\begin{align}
& -(R_{11,m} \cos \omega t - I_{11,m} \sin \omega t )=|{q}_{m,a,a}| \{ R_{12,m} \cos (\omega t + \varphi_{m,a,a} ) - I_{12,m} \sin (\omega t + \varphi_{m,a,a} ) \} \\ \\
\Rightarrow \quad &\sqrt{{R_{11,m}}^2 + {I_{11,m}}^2} \cos \Bigl( \omega t + \tan^{-1} \frac{I_{11,m}}{R_{11,m}} + \pi \Bigr) = |{q}_{m,a,a}|\sqrt{{R_{12,m}}^2 + {I_{12,m}}^2} \cos \Bigl( \omega t + \varphi_{m,a,a} + \tan^{-1} \frac{I_{12,m}}{R_{12,m}} \Bigr) \\
\Rightarrow \quad &|{q}_{m,a,a}| = \frac{\sqrt{{R_{11,m}}^2 + {I_{11,m}}^2}}{\sqrt{{R_{12,m}}^2 + {I_{12,m}}^2}}, \qquad \varphi_{m,a,a} = \tan^{-1} \frac{I_{11,m}}{R_{11,m}} - \tan^{-1} \frac{I_{12,m}}{R_{12,m}} + \pi  \qquad (i) \\
\end{align} $$

を得る。また、式$(h)$第2式から、

$$ \begin{align} 
& - |{q}_{m,a,b}| \cos (\omega t + \varphi_{m,a,b}) = \frac{R_{12,m}}{{R_{12,m}}^2 + {I_{12,m}}^2}\cos \omega t + \frac{I_{12,m}}{{R_{12,m}}^2 + {I_{12,m}}^2}\sin \omega t \\ \\
\Rightarrow \quad & |{q}_{m,a,b}| \cos (\omega t + \varphi_{m,a,b} + \pi) = \frac{1}{\sqrt{{R_{12,m}}^2 + {I_{12,m}}^2}} \;\cos \Bigl( \omega t - \tan^{-1} \frac{I_{12,m}}{R_{12,m}} \Bigr) \\ 
\Rightarrow \quad & |{q}_{m,a,b}| = \frac{1}{\sqrt{{R_{12,m}}^2 + {I_{12,m}}^2}}, \qquad \varphi_{m,a,b} = - \tan^{-1} \frac{I_{12,m}}{R_{12,m}} - \pi  \qquad (j)\\
\end{align} $$ 

を得る。よって、$\hat{\theta}_{m,a} = \cos \omega t$、$\hat{\theta}_{m,b} = 0\:$の$a$側での単位温度励振の場合の熱流応答は、

$$
\left\{
\begin{array}{ll}
\hat{q}_{m,a,a} = \frac{\sqrt{{R_{11,m}}^2 + {I_{11,m}}^2}}{\sqrt{{R_{12,m}}^2 + {I_{12,m}}^2}} \;\cos \Bigl( \omega t +  \tan^{-1} \frac{I_{11,m}}{R_{11,m}} - \tan^{-1} \frac{I_{12,m}}{R_{12,m}} + \pi \Bigr)
\\
\hat{q}_{m,a,b} = \frac{1}{\sqrt{{R_{12,m}}^2 + {I_{12,m}}^2}} \; \cos \Bigl( \omega t - \tan^{-1} \frac{I_{12,m}}{R_{12,m}} - \pi \Bigr)
\end{array}
\right.  \qquad (k)
$$

で求まることになる。また、これらの熱流応答$\; \hat{q}_{m,a,*} = |{q}_{m,a,*}| \cos \varphi_{m,a,*} \cos \omega t - |{q}_{m,a,*}| \sin \varphi_{m,a,*} \sin \omega t \; $の係数は以下となる。

$$
\left\{
\begin{array}{ll}
|{q}_{m,a,a}| \cos \varphi_{m,a,a} = \frac{\sqrt{{R_{11,m}}^2 + {I_{11,m}}^2}}{\sqrt{{R_{12,m}}^2 + {I_{12,m}}^2}} \cos \Bigl( \tan^{-1} \frac{I_{11,m}}{R_{11,m}} - \tan^{-1} \frac{I_{12,m}}{R_{12,m}} + \pi \Bigr)
\\
|{q}_{m,a,a}| \sin \varphi_{m,a,a} = \frac{\sqrt{{R_{11,m}}^2 + {I_{11,m}}^2}}{\sqrt{{R_{12,m}}^2 + {I_{12,m}}^2}} \sin \Bigl( \tan^{-1} \frac{I_{11,m}}{R_{11,m}} - \tan^{-1} \frac{I_{12,m}}{R_{12,m}} + \pi \Bigr)
\\
|{q}_{m,a,b}| \cos \varphi_{m,a,b} = \frac{1}{\sqrt{{R_{12,m}}^2 + {I_{12,m}}^2}} \cos \Bigl( - \tan^{-1} \frac{I_{12,m}}{R_{12,m}} - \pi \Bigr)
\\
|{q}_{m,a,b}| \sin \varphi_{m,a,b} = \frac{1}{\sqrt{{R_{12,m}}^2 + {I_{12,m}}^2}} \sin \Bigl( - \tan^{-1} \frac{I_{12,m}}{R_{12,m}} - \pi \Bigr)
\end{array}
\right.  \qquad (l)
$$

##### c) b側が励振する際の熱流応答

躯体$m$の裏面側($b$側)の温度が温度差係数$H_m$の振幅および$\psi_m' \:( \; = 2 \pi \: \big/ \: (T_{\psi,m} \times 3600) \: )$の位相のずれをもって変動する時の周波数応答を求める。

$\hat{\theta}_{m,b} = (1 - H_m) \cos (\omega t + \psi_m' )$、$\hat{\theta}_{m,a} = 0\:$で$b$側で温度励振する場合を考える。式$(c)$、$(d)$の実数部のみ考慮し、$b$側励振に対する$a$側応答$\hat{q}_{m,b,a} = |{q}_{m,b,a}|e^{ j (\omega t + \varphi_{m,b,a} ) }$、$b$側応答$\hat{q}_{m,b,b} = |{q}_{m,b,b}|e^{ j (\omega t + \varphi_{m,b,b} ) }$を求める。

$|{\theta}_{m,b}|=1 - H_m$、$\psi_{m,b}=\psi_m' $、$|{\theta}_{m,a}|=0$を、式$(g)$第1式に代入して、

$$\begin{align}
& (1 - H_m) \cos (\omega t + \psi_m' )  = |{q}_{m,b,a}| \{ R_{12,m} \cos (\omega t + \varphi_{m,b,a} ) - I_{12,m} \sin (\omega t + \varphi_{m,b,a} ) \} \\
\Rightarrow \quad & (1 - H_m) \cos (\omega t + \psi_m' )  = |{q}_{m,b,a}|\sqrt{{R_{12,m}}^2 + {I_{12,m}}^2} \cos \Bigl( \omega t + \varphi_{m,b,a} + \tan^{-1} \frac{I_{12,m}}{R_{12,m}} \Bigr) \\
\Rightarrow \quad & |{q}_{m,b,a}| = \frac{1 - H_m}{\sqrt{{R_{12,m}}^2 + {I_{12,m}}^2}}, \qquad \varphi_{m,b,a} =  \psi_m' -  \tan^{-1} \frac{I_{12,m}}{R_{12,m}} \qquad (m) \\
\end{align} $$

を得る。また、式$(h)$第2式から、

$$ \begin{align}
& - |{q}_{m,b,b}| \{R_{12,m}\cos (\omega t + \varphi_{m,b,b} ) - I_{12,m}\cos (\omega t + \varphi_{m,b,b} ) \} = -(1 - H_m) \{ R_{22,m} \cos (\omega t + \psi_m' ) - I_{22,m} \sin (\omega t + \psi_m' ) \} \\
\Rightarrow \quad & |{q}_{m,b,b}| \sqrt{{R_{12,m}}^2 + {I_{12,m}}^2} \cos \Bigl( \omega t + \varphi_{m,b,b} +  \tan^{-1} \frac{I_{12,m}}{R_{12,m}} \Bigr) = (1 - H_m) \sqrt{{R_{22,m}}^2 + {I_{22,m}}^2} \cos \Bigl( \omega t + \psi_m' +  \tan^{-1} \frac{I_{22,m}}{R_{22,m}} \Bigr) \\
\Rightarrow \quad & |{q}_{m,b,b}| = \frac{(1 - H_m) \sqrt{{R_{22,m}}^2 + {I_{22,m}}^2}}{\sqrt{{R_{12,m}}^2 + {I_{12,m}}^2}}, \qquad \varphi_{m,b,b} = \psi_m' -  \tan^{-1} \frac{I_{12,m}}{R_{12,m}} + \tan^{-1} \frac{I_{22,m}}{R_{22,m}} \qquad (n)\\
\end{align} $$ 

を得る。よって、$\hat{\theta}_{m,b} = (1 - H_m) \cos (\omega t + \psi_m' )$、$\hat{\theta}_{m,a} = 0\:$で$b$側で温度励振する場合の熱流応答は、

$$
\left\{
\begin{array}{l}
\hat{q}_{m,b,a} = \frac{1 - H_m}{\sqrt{{R_{12,m}}^2 + {I_{12,m}}^2}} \;\cos \Bigl( \omega t + \psi_m' -  \tan^{-1} \frac{I_{12,m}}{R_{12,m}} \Bigr)
\\
\hat{q}_{m,b,b} = \frac{(1 - H_m) \sqrt{{R_{22,m}}^2 + {I_{22,m}}^2}}{\sqrt{{R_{12,m}}^2 + {I_{12,m}}^2}} \; \cos \Bigl( \omega t + \psi_m' -  \tan^{-1} \frac{I_{12,m}}{R_{12,m}} + \tan^{-1} \frac{I_{22,m}}{R_{22,m}} \Bigr)
\end{array}
\right.  \qquad (o)
$$

で求まることになる。ここで、式$(o)$第1式で、$H_m=0$、$\psi_m'=0$とし、熱流の向きを反転して考えると、式$(k)$第2式と同一となり、貫流応答の相反則が確認される。

また、これらの熱流応答$\; \hat{q}_{m,b,*} = |{q}_{m,b,*}| \cos \varphi_{m,b,*} \cos \omega t - |{q}_{m,b,*}| \sin \varphi_{m,b,*} \sin \omega t \; $の係数は以下となる。

$$
\left\{
\begin{array}{ll}
|{q}_{m,b,a}| \cos \varphi_{m,b,a} = \frac{1 - H_m}{\sqrt{{R_{12,m}}^2 + {I_{12,m}}^2}} \cos \Bigl( \psi_m' -  \tan^{-1} \frac{I_{12,m}}{R_{12,m}} \Bigr) 
\\
|{q}_{m,b,a}| \sin \varphi_{m,b,a} = \frac{1 - H_m}{\sqrt{{R_{12,m}}^2 + {I_{12,m}}^2}} \sin \Bigl( \psi_m' -  \tan^{-1} \frac{I_{12,m}}{R_{12,m}} \Bigr) 
\\
|{q}_{m,b,b}| \cos \varphi_{m,b,b} = \frac{(1 - H_m) \sqrt{{R_{22,m}}^2 + {I_{22,m}}^2}}{\sqrt{{R_{12,m}}^2 + {I_{12,m}}^2}} \cos \Bigl( \psi_m' -  \tan^{-1} \frac{I_{12,m}}{R_{12,m}} + \tan^{-1} \frac{I_{22,m}}{R_{22,m}} \Bigr)
\\
|{q}_{m,b,b}| \sin \varphi_{m,b,b} = \frac{(1 - H_m) \sqrt{{R_{22,m}}^2 + {I_{22,m}}^2}}{\sqrt{{R_{12,m}}^2 + {I_{12,m}}^2}} \sin \Bigl( \psi_m' -  \tan^{-1} \frac{I_{12,m}}{R_{12,m}} + \tan^{-1} \frac{I_{22,m}}{R_{22,m}} \Bigr)
\\
\end{array}
\right.  \qquad (p)
$$


##### d) a側、b側が温度励振する時のa側、b側の熱流応答

$a$側の温度が$\hat{\theta}_{m,a} = \cos \omega t$、$b$側の温度が$\hat{\theta}_{m,b} = (1 - H_m) \cos (\omega t + \psi_m' )$で変動するときの、$a$側の熱流$\hat{q}_{m,a}$、$b$側の熱流$\hat{q}_{m,b}$：

$$
\left\{
\begin{array}{ll}
\hat{q}_{m,a} = \hat{q}_{m,a,a}+\hat{q}_{m,b,a} = |{q}_{m,a}| \cos \varphi_{m,a} \cos \omega t - |{q}_{m,a}| \sin \varphi_{m,a}  \sin \omega t
\\
\hat{q}_{m,b} = \hat{q}_{m,a,b}+\hat{q}_{m,b,b} = |{q}_{m,b}| \cos \varphi_{m,b} \cos \omega t - |{q}_{m,b}| \sin \varphi_{m,b}  \sin \omega t
\end{array}
\right.  \qquad (q)
$$

$$
\left\{
\begin{array}{ll}
|{q}_{m,a}| \cos \varphi_{m,a} = |{q}_{m,a,a}| \cos \varphi_{m,a,a}+|{q}_{m,b,a}| \cos \varphi_{m,b,a}\\
|{q}_{m,a}| \sin \varphi_{m,a} = |{q}_{m,a,a}| \sin \varphi_{m,a,a}+|{q}_{m,b,a}| \sin \varphi_{m,b,a}\\
|{q}_{m,b}| \cos \varphi_{m,b} = |{q}_{m,a,b}| \cos \varphi_{m,a,b}+|{q}_{m,b,b}| \cos \varphi_{m,b,b}\\
|{q}_{m,b}| \sin \varphi_{m,b} = |{q}_{m,a,b}| \sin \varphi_{m,a,b}+|{q}_{m,b,b}| \sin \varphi_{m,b,b}\\
\end{array}
\right.  \qquad (r)
$$


##### e) 熱流の定常成分

$a$側の温度が$\hat{\theta}_{m,a} = \cos \omega t$、$b$側の温度が$\hat{\theta}_{m,b} = (1 - H_m) \cos (\omega t + \psi_m' )$で変動するときの熱流の定常成分$\hat{q}_{s,m}$：

$$\begin{align}
\hat{q}_{s,m} &= U_m (\hat{\theta}_{m,a}-\hat{\theta}_{m,b}) \\
&= U_m \{ \cos \omega t - (1 - H_m) \cos (\omega t + \psi_m' ) \} \\
&= U_m (1- \cos \psi_m'+ H_m \cos \psi_m' ) \cos \omega t + U_m (1 - H_m) \sin \psi_m' \sin \omega t   \qquad (s)\\
\end{align} $$

#### (7) 有効熱容量

##### a) 有効熱容量について

ISO 13786では、室内側($a$側)の単位面積あたりの有効熱容量$\kappa_1 \:[J/(m^2･K)]$を、下式で求めている。

$$\kappa_1 = \frac{T}{2 \pi} \Biggl| \frac{Z_{11}-1}{Z_{12}} \Biggr| = \frac{1}{\omega} \Biggl| \frac{Z_{11}-1}{Z_{12}} \Biggr| \qquad (22')$$

この有効熱容量$\kappa_1$は、$a$側が単位振幅で温度振動するときの$a$側熱流応答と$b$側熱流応答の差の時間積分(すなわち、躯体の蓄熱量の時間変化)の振幅を意味している。

##### b) 有効熱容量算定のための周波数応答の積算

対象とする空間についての有効熱容量${C}_{i}$、室内側吸熱有効熱容量${C}_{Ai}$、貫流有効熱容量${C}_{T}$を計算するために、躯体$m$($=1$～$M$)について、`Wtype`にもとづいて熱流を積算する。

室内側吸熱有効熱容量${C}_{Ai}$を求めるための熱流差$\hat{Q}_{Ai}$は次式で求まる。

$$\hat{Q}_{Ai} = |{Q}_{Ai}| \cos \varphi_{Ai} \cos \omega t - |{Q}_{Ai}| \sin \varphi_{Ai} \sin \omega t = \sum_{m=1}^{M} A_m \: \bigl\{ (\hat{q}_{m,a} - \hat{q}_{s,m} ) + \delta_{wtype=1} ( \hat{q}_{s,m} - \hat{q}_{m,b} ) \bigl\} \qquad (t)$$
$$\begin{align}
|{Q}_{Ai}| \cos \varphi_{Ai} = \sum_{m=1}^{M} A_m \:\bigl[ & |{q}_{m,a}| \cos \varphi_{m,a} - U_m (1- \cos \psi_m'+ H_m \cos \psi_m' ) \\
&+ \delta_{wtype=1} \{ U_m (1- \cos \psi_m'+ H_m \cos \psi_m' ) - |{q}_{m,b}| \cos \varphi_{m,b} \} \bigr] \qquad (t1)
\end{align} $$
$$\begin{align}
|{Q}_{Ai}| \sin \varphi_{Ai} = \sum_{m=1}^{M} A_m \:\bigl[ & |{q}_{m,a}| \sin \varphi_{m,a} - U_m (1 - H_m) \sin \psi_m' \\
&+ \delta_{wtype=1} \{ U_m (1 - H_m) \sin \psi_m' - |{q}_{m,b}| \sin \varphi_{m,b} \} \bigr] \qquad (t2)
\end{align} $$

貫流有効熱容量${C}_{T}$を求めるための熱流差$\hat{Q}_{T}$は次式で求まる。

$$\hat{Q}_{T} = |{Q}_{T}| \cos \varphi_{T} \cos \omega t - |{Q}_{T}| \sin \varphi_{T} \sin \omega t =
\sum_{m=1}^{M} \delta_{wtype=0} \: A_m \:(\hat{q}_{s,m} - \hat{q}_{m,b}) \bigl\} \qquad (u)$$
$$
|{Q}_{T}| \cos \varphi_{T} = \sum_{m=1}^{M} \delta_{wtype=0} \: A_m \:\{ U_m (1- \cos \psi_m'+ H_m \cos \psi_m' ) - |{q}_{m,b}| \cos \varphi_{m,b} \} \qquad (u1)
$$
$$
|{Q}_{T}| \sin \varphi_{T} = \sum_{m=1}^{M} \delta_{wtype=0} \: A_m \:\{ U_m (1 - H_m) \sin \psi_m' - |{q}_{m,b}| \sin \varphi_{m,b}  \} \qquad (u2)
$$

有効熱容量${C}_{i}$を求めるための熱流差$\hat{Q}_{i}$は次式で求まる。

$$\hat{Q}_{i} = |{Q}_{i}| \cos \varphi_{i} \cos \omega t - |{Q}_{i}| \sin \varphi_{i} \sin \omega t = \sum_{m=1}^{M} A_m \: \bigl\{ (\hat{q}_{m,a} - \hat{q}_{s,m} ) + \delta_{wtype \neq 2} ( \hat{q}_{s,m} - \hat{q}_{m,b} ) \bigl\} \qquad (v)$$
$$\begin{align}
|{Q}_{i}| \cos \varphi_{i} = \sum_{m=1}^{M} A_m \:\bigl[ & |{q}_{m,a}| \cos \varphi_{m,a} - U_m (1- \cos \psi_m'+ H_m \cos \psi_m' ) \\
&+ \delta_{wtype \neq 2} \{ U_m (1- \cos \psi_m'+ H_m \cos \psi_m' ) - |{q}_{m,b}| \cos \varphi_{m,b} \} \bigr] \qquad (v1)
\end{align} $$
$$\begin{align}
|{Q}_{i}| \sin \varphi_{i} = \sum_{m=1}^{M} A_m \:\bigl[ & |{q}_{m,a}| \sin \varphi_{m,a} - U_m (1 - H_m) \sin \psi_m' \\
&+ \delta_{wtype \neq 2} \{ U_m (1 - H_m) \sin \psi_m' - |{q}_{m,b}| \sin \varphi_{m,b} \} \bigr] \qquad (v2)
\end{align} $$

$\delta_{wtype=0}\:$は、`Wtype`$=0$で$1$を、それ以外の`Wtype`で$0$をとる。$\delta_{wtype=1}\:$は、`Wtype`$=1$で$1$を、それ以外の`Wtype`で$0$をとる。また、$\delta_{wtype \neq 2}\:$は、`Wtype`$=0$、`Wtype`$=1$で$1$を、`Wtype`$=2$で$0$をとる。


##### c) 有効熱容量の算定

室内側吸熱有効熱容量${C}_{Ai}$は、以下で求まる。

$$ \begin{align} 
C_{Ai} &= \Bigl| \int \hat{Q}_{Ai} \;dt \; \Bigr| \\
&= \Bigl| \int ( |{Q}_{Ai}| \cos \varphi_{Ai} \cos \omega t - |{Q}_{Ai}| \sin \varphi_{Ai} \sin \omega t ) 
\;dt \; \Bigr| \\
&= \frac{1}{\omega} \: \bigl| \: ( |{Q}_{Ai}| \cos \varphi_{Ai} \sin \omega t + |{Q}_{Ai}| \sin \varphi_{Ai} \cos \omega t ) \; \bigr| \\
&= \frac{1}{\omega} \: \Bigl| \: \sqrt{ |{Q}_{Ai}|^2 \cos^2 \varphi_{Ai} + |{Q}_{Ai}|^2 \sin^2 \varphi_{Ai} } \cos \Bigl( \omega t - tan ^ {-1} \frac{\cos \varphi_{Ai}}{\sin \varphi_{Ai}} \Bigr) \Bigr| \\
&= |{Q}_{Ai}| \big/ \omega  \qquad (w) \\
\end{align} $$ 

同様に、貫流有効熱容量${C}_{T}$、有効熱容量${C}_{i}$は、以下で求まる。

$$C_{T} = \Bigl| \int \hat{Q}_{T} \;dt \; \Bigr| = \frac{1}{\omega} \: \sqrt{ |{Q}_{T}|^2 \cos^2 \varphi_{T} + |{Q}_{T}|^2 \sin^2 \varphi_{T} } = |{Q}_{T}| \big/ \omega  \qquad (x) $$ 
$$C_{i} = \Bigl| \int \hat{Q}_{i} \;dt \; \Bigr| = \frac{1}{\omega} \: \sqrt{ |{Q}_{i}|^2 \cos^2 \varphi_{i} + |{Q}_{i}|^2 \sin^2 \varphi_{i} } = |{Q}_{i}| \big/ \omega  \qquad (y) $$ 

有効熱容量は周期$T$によって値がかわる。本稿では周期$T$は$24$時間としている。


